In [ ]:
#@title 1.1 Check GPU Status
import subprocess
simple_nvidia_smi_display = False#@param {type:"boolean"}
nvidiasmi_output=""
if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8') 
  print(nvidiasmi_output) 
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print('heloooo',nvidiasmi_ecc_note)
gpu_name=nvidiasmi_output.split('|   0')[1].split('Off')[0].strip()
print(gpu_name)

<a href="https://colab.research.google.com/github/tjthejuggler/Pop-Diffusion/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disco Diffusion v5.3 - Now with Symmetry

In case of confusion, Disco is the name of this notebook edit. The diffusion model in use is Katherine Crowson's fine-tuned 512x512 model

For issues, join the [Disco Diffusion Discord](https://discord.gg/msEZBy4HxA) or message us on twitter at [@somnai_dreams](https://twitter.com/somnai_dreams) or [@gandamu](https://twitter.com/gandamu_ml)

### Credits & Changelog ⬇️

#### Credits

Original notebook by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses either OpenAI's 256x256 unconditional ImageNet or Katherine Crowson's fine-tuned 512x512 diffusion model (https://github.com/openai/guided-diffusion), together with CLIP (https://github.com/openai/CLIP) to connect text prompts with images.

Modified by Daniel Russell (https://github.com/russelldc, https://twitter.com/danielrussruss) to include (hopefully) optimal params for quick generations in 15-100 timesteps rather than 1000, as well as more robust augmentations.

Further improvements from Dango233 and nsheppard helped improve the quality of diffusion in general, and especially so for shorter runs like this notebook aims to achieve.

Vark added code to load in multiple Clip models at once, which all prompts are evaluated against, which may greatly improve accuracy.

The latest zoom, pan, rotation, and keyframes features were taken from Chigozie Nri's VQGAN Zoom Notebook (https://github.com/chigozienri, https://twitter.com/chigozienri)

Advanced DangoCutn Cutout method is also from Dango223.

--

Disco:

Somnai (https://twitter.com/Somnai_dreams) added Diffusion Animation techniques, QoL improvements and various implementations of tech and techniques, mostly listed in the changelog below.

3D animation implementation added by Adam Letts (https://twitter.com/gandamu_ml) in collaboration with Somnai. Creation of disco.py and ongoing maintenance.

Turbo feature by Chris Allen (https://twitter.com/zippy731)

Improvements to ability to run on local systems, Windows support, and dependency installation by HostsServer (https://twitter.com/HostsServer)

VR Mode by Tom Mason (https://twitter.com/nin_artificial)

Horizontal and Vertical symmetry functionality by nsheppard. Symmetry transformation_steps by huemin (https://twitter.com/huemin_art). Symmetry integration into Disco Diffusion by Dmitrii Tochilkin (https://twitter.com/cut_pow).

#### License

Licensed under the MIT License

Copyright (c) 2021 Katherine Crowson 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

--

MIT License

Copyright (c) 2019 Intel ISL (Intel Intelligent Systems Lab)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

--

Licensed under the MIT License

Copyright (c) 2021 Maxwell Ingham

Copyright (c) 2022 Adam Letts 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

#### Changelog

In [ ]:
#@title <- View Changelog
skip_for_run_all = True #@param {type: 'boolean'}

if skip_for_run_all == False:
  print(
      '''
  v1 Update: Oct 29th 2021 - Somnai

      QoL improvements added by Somnai (@somnai_dreams), including user friendly UI, settings+prompt saving and improved google drive folder organization.

  v1.1 Update: Nov 13th 2021 - Somnai

      Now includes sizing options, intermediate saves and fixed image prompts and perlin inits. unexposed batch option since it doesn't work

  v2 Update: Nov 22nd 2021 - Somnai

      Initial addition of Katherine Crowson's Secondary Model Method (https://colab.research.google.com/drive/1mpkrhOjoyzPeSWy2r7T8EYRaU7amYOOi#scrollTo=X5gODNAMEUCR)

      Noticed settings were saving with the wrong name so corrected it. Let me know if you preferred the old scheme.

  v3 Update: Dec 24th 2021 - Somnai

      Implemented Dango's advanced cutout method

      Added SLIP models, thanks to NeuralDivergent

      Fixed issue with NaNs resulting in black images, with massive help and testing from @Softology

      Perlin now changes properly within batches (not sure where this perlin_regen code came from originally, but thank you)

  v4 Update: Jan 2021 - Somnai

      Implemented Diffusion Zooming

      Added Chigozie keyframing

      Made a bunch of edits to processes
  
  v4.1 Update: Jan  14th 2021 - Somnai

      Added video input mode

      Added license that somehow went missing

      Added improved prompt keyframing, fixed image_prompts and multiple prompts

      Improved UI

      Significant under the hood cleanup and improvement

      Refined defaults for each mode

      Added latent-diffusion SuperRes for sharpening

      Added resume run mode

  v4.9 Update: Feb 5th 2022 - gandamu / Adam Letts

      Added 3D

      Added brightness corrections to prevent animation from steadily going dark over time

  v4.91 Update: Feb 19th 2022 - gandamu / Adam Letts

      Cleaned up 3D implementation and made associated args accessible via Colab UI elements

  v4.92 Update: Feb 20th 2022 - gandamu / Adam Letts

      Separated transform code

  v5.01 Update: Mar 10th 2022 - gandamu / Adam Letts

      IPython magic commands replaced by Python code

  v5.1 Update: Mar 30th 2022 - zippy / Chris Allen and gandamu / Adam Letts

      Integrated Turbo+Smooth features from Disco Diffusion Turbo -- just the implementation, without its defaults.

      Implemented resume of turbo animations in such a way that it's now possible to resume from different batch folders and batch numbers.

      3D rotation parameter units are now degrees (rather than radians)

      Corrected name collision in sampling_mode (now diffusion_sampling_mode for plms/ddim, and sampling_mode for 3D transform sampling)

      Added video_init_seed_continuity option to make init video animations more continuous

  v5.1 Update: Apr 4th 2022 - MSFTserver aka HostsServer

      Removed pytorch3d from needing to be compiled with a lite version specifically made for Disco Diffusion

      Remove Super Resolution

      Remove SLIP Models

      Update for crossplatform support

  v5.2 Update: Apr 10th 2022 - nin_artificial / Tom Mason

      VR Mode

  v5.3 Update: Jun 10th 2022 - nsheppard, huemin, cut_pow

      Horizontal and Vertical symmetry

      Addition of ViT-L/14@336px model (requires high VRAM)
    '''
  )

### Tutorial

**Diffusion settings (Defaults are heavily outdated)**
---
Disco Diffusion is complex, and continually evolving with new features.  The most current documentation on on Disco Diffusion settings can be found in the unofficial guidebook:

[Zippy's Disco Diffusion Cheatsheet](https://docs.google.com/document/d/1l8s7uS2dGqjztYSjPpzlmXLjl5PM3IGkRWI3IiCuK7g/edit)

We also encourage users to join the [Disco Diffusion User Discord](https://discord.gg/XGZrFFCRfN) to learn from the active user community.

This section below is outdated as of v2

Setting | Description | Default
--- | --- | ---
**Your vision:**
`text_prompts` | A description of what you'd like the machine to generate. Think of it like writing the caption below your image on a website. | N/A
`image_prompts` | Think of these images more as a description of their contents. | N/A
**Image quality:**
`clip_guidance_scale`  | Controls how much the image should look like the prompt. | 1000
`tv_scale` | Controls the smoothness of the final output. | 150
`range_scale` | Controls how far out of range RGB values are allowed to be. | 150
`sat_scale` | Controls how much saturation is allowed. From nshepperd's JAX notebook. | 0
`cutn` | Controls how many crops to take from the image. | 16
`cutn_batches` | Accumulate CLIP gradient from multiple batches of cuts. | 2
**Init settings:**
`init_image` | URL or local path | None
`init_scale` | This enhances the effect of the init image, a good value is 1000 | 0
`skip_steps` | Controls the starting point along the diffusion timesteps | 0
`perlin_init` | Option to start with random perlin noise | False
`perlin_mode` | ('gray', 'color') | 'mixed'
**Advanced:**
`skip_augs` | Controls whether to skip torchvision augmentations | False
`randomize_class` | Controls whether the imagenet class is randomly changed each iteration | True
`clip_denoised` | Determines whether CLIP discriminates a noisy or denoised image | False
`clamp_grad` | Experimental: Using adaptive clip grad in the cond_fn | True
`seed`  | Choose a random seed and print it at end of run for reproduction | random_seed
`fuzzy_prompt` | Controls whether to add multiple noisy prompts to the prompt losses | False
`rand_mag` | Controls the magnitude of the random noise | 0.1
`eta` | DDIM hyperparameter | 0.5

..

**Model settings**
---

Setting | Description | Default
--- | --- | ---
**Diffusion:**
`timestep_respacing` | Modify this value to decrease the number of timesteps. | ddim100
`diffusion_steps` || 1000
**Diffusion:**
`clip_models` | Models of CLIP to load. Typically the more, the better but they all come at a hefty VRAM cost. | ViT-B/32, ViT-B/16, RN50x4

## 1. Set Up

In [ ]:
#@title 1.2 Prepare Folders
import subprocess, os, sys, ipykernel

def gitclone(url):
  res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipi(modulestr):
  res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipie(modulestr):
  res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def wget(url, outputdir):
  res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    #@markdown If you connect your Google Drive, you can save the final image of each run on your drive.
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

mounted_at_beginning = False

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        mounted_at_beginning = True
        root_path = '/content/drive/MyDrive/AI/Disco_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_images'
createPath(initDirPath)
outDirPath = f'{root_path}/images_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

# libraries = f'{root_path}/libraries'
# createPath(libraries)


## 4. Install and Import Dependencies

In [ ]:
#@title ### 1.3 Install and import dependencies

import pathlib, shutil, os, sys
import time

if not is_colab:
  # If running locally, there's a good chance your env will need this in order to not crash upon np.matmul() or similar operations.
  os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

PROJECT_DIR = os.path.abspath(os.getcwd())
USE_ADABINS = True

if is_colab:
  if google_drive is not True:
    root_path = f'/content'
    model_path = '/content/models' 
else:
  root_path = os.getcwd()
  model_path = f'{root_path}/models'

model_256_downloaded = False
model_512_downloaded = False
model_secondary_downloaded = False

multipip_res = subprocess.run(['pip', 'install', 'lpips', 'datetime', 'timm', 'ftfy', 'einops', 'pytorch-lightning', 'omegaconf'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(multipip_res)

if is_colab:
  subprocess.run(['apt', 'install', 'imagemagick'], stdout=subprocess.PIPE).stdout.decode('utf-8')

try:
  from CLIP import clip
except:
  if not os.path.exists("CLIP"):
    gitclone("https://github.com/openai/CLIP")
  sys.path.append(f'{PROJECT_DIR}/CLIP')

try:
  from guided_diffusion.script_util import create_model_and_diffusion
except:
  if not os.path.exists("guided-diffusion"):
    #gitclone("https://github.com/crowsonkb/guided-diffusion")
    gitclone("https://github.com/kostarion/guided-diffusion")
  sys.path.append(f'{PROJECT_DIR}/guided-diffusion')

try:
  from resize_right import resize
except:
  if not os.path.exists("ResizeRight"):
    gitclone("https://github.com/assafshocher/ResizeRight.git")
  sys.path.append(f'{PROJECT_DIR}/ResizeRight')

try:
  import py3d_tools
except:
  if not os.path.exists('pytorch3d-lite'):
    gitclone("https://github.com/MSFTserver/pytorch3d-lite.git")
  sys.path.append(f'{PROJECT_DIR}/pytorch3d-lite')

try:
  from midas.dpt_depth import DPTDepthModel
except:
  if not os.path.exists('MiDaS'):
    gitclone("https://github.com/isl-org/MiDaS.git")
  if not os.path.exists('MiDaS/midas_utils.py'):
    shutil.move('MiDaS/utils.py', 'MiDaS/midas_utils.py')
  if not os.path.exists(f'{model_path}/dpt_large-midas-2f21e586.pt'):
    wget("https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt", model_path)
  sys.path.append(f'{PROJECT_DIR}/MiDaS')

try:
  sys.path.append(PROJECT_DIR)
  import disco_xform_utils as dxf
except:
  if not os.path.exists("disco-diffusion"):
    gitclone("https://github.com/alembics/disco-diffusion.git")
  if os.path.exists('disco_xform_utils.py') is not True:
    shutil.move('disco-diffusion/disco_xform_utils.py', 'disco_xform_utils.py')
  sys.path.append(PROJECT_DIR)

import torch
from dataclasses import dataclass
from functools import partial
import cv2
import pandas as pd
import gc
import io
import math
import timm
from IPython import display
import lpips
from PIL import Image, ImageOps
import requests
from glob import glob
import json
from types import SimpleNamespace
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from tqdm.notebook import tqdm
from CLIP import clip
from resize_right import resize
from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import random
from ipywidgets import Output
import hashlib
from functools import partial
if is_colab:
  os.chdir('/content')
  from google.colab import files
else:
  os.chdir(f'{PROJECT_DIR}')
from IPython.display import Image as ipyimg
from numpy import asarray
from einops import rearrange, repeat
import torch, torchvision
import time
from omegaconf import OmegaConf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# AdaBins stuff
if USE_ADABINS:
  try:
    from infer import InferenceHelper
  except:
    if os.path.exists("AdaBins") is not True:
      gitclone("https://github.com/shariqfarooq123/AdaBins.git")
    if not os.path.exists(f'{PROJECT_DIR}/pretrained/AdaBins_nyu.pt'):
      createPath(f'{PROJECT_DIR}/pretrained')
      wget("https://cloudflare-ipfs.com/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt", f'{PROJECT_DIR}/pretrained')
    sys.path.append(f'{PROJECT_DIR}/AdaBins')
  from infer import InferenceHelper
  MAX_ADABINS_AREA = 500000

import torch
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
device = DEVICE # At least one of the modules expects this name..

if torch.cuda.get_device_capability(DEVICE) == (8,0): ## A100 fix thanks to Emad
  print('Disabling CUDNN for A100 gpu', file=sys.stderr)
  torch.backends.cudnn.enabled = False

## 5. Define Midas functions

In [ ]:
#@title ### 1.4 Define Midas functions

from midas.dpt_depth import DPTDepthModel
from midas.midas_net import MidasNet
from midas.midas_net_custom import MidasNet_small
from midas.transforms import Resize, NormalizeImage, PrepareForNet

# Initialize MiDaS depth model.
# It remains resident in VRAM and likely takes around 2GB VRAM.
# You could instead initialize it for each frame (and free it after each frame) to save VRAM.. but initializing it is slow.
default_models = {
    "midas_v21_small": f"{model_path}/midas_v21_small-70d6b9c8.pt",
    "midas_v21": f"{model_path}/midas_v21-f6b98070.pt",
    "dpt_large": f"{model_path}/dpt_large-midas-2f21e586.pt",
    "dpt_hybrid": f"{model_path}/dpt_hybrid-midas-501f0c75.pt",
    "dpt_hybrid_nyu": f"{model_path}/dpt_hybrid_nyu-2ce69ec7.pt",}

def init_midas_depth_model(midas_model_type="dpt_large", optimize=True):
    midas_model = None
    net_w = None
    net_h = None
    resize_mode = None
    normalization = None

    print(f"Initializing MiDaS '{midas_model_type}' depth model...")
    # load network
    midas_model_path = default_models[midas_model_type]

    if midas_model_type == "dpt_large": # DPT-Large
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitl16_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode = "minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid": #DPT-Hybrid
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid_nyu": #DPT-Hybrid-NYU
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "midas_v21":
        midas_model = MidasNet(midas_model_path, non_negative=True)
        net_w, net_h = 384, 384
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    elif midas_model_type == "midas_v21_small":
        midas_model = MidasNet_small(midas_model_path, features=64, backbone="efficientnet_lite3", exportable=True, non_negative=True, blocks={'expand': True})
        net_w, net_h = 256, 256
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    else:
        print(f"midas_model_type '{midas_model_type}' not implemented")
        assert False

    midas_transform = T.Compose(
        [
            Resize(
                net_w,
                net_h,
                resize_target=None,
                keep_aspect_ratio=True,
                ensure_multiple_of=32,
                resize_method=resize_mode,
                image_interpolation_method=cv2.INTER_CUBIC,
            ),
            normalization,
            PrepareForNet(),
        ]
    )

    midas_model.eval()
    
    if optimize==True:
        if DEVICE == torch.device("cuda"):
            midas_model = midas_model.to(memory_format=torch.channels_last)  
            midas_model = midas_model.half()

    midas_model.to(DEVICE)

    print(f"MiDaS '{midas_model_type}' depth model initialized.")
    return midas_model, midas_transform, net_w, net_h, resize_mode, normalization

## **1.5 Define necessary functions**

In [ ]:
# https://gist.github.com/adefossez/0646dbe9ed4005480a2407c62aac8869

import py3d_tools as p3dT
import disco_xform_utils as dx
import os


stats={
  
    "gpu_name":gpu_name,
    "frame_durations": [],
    "antarctic_prompt_durations": [],
    "object_detection_durations": [],
    "average_frame_duration": '0',
    "average_antarctic_prompt_duration": '0',
    "average_object_detection_duration": '0',
    "interpolation_duration": '0',

}


  
antarctic_prompt_durations = []

def interp(t):
    return 3 * t**2 - 2 * t ** 3

def perlin(width, height, scale=10, device=None):
    gx, gy = torch.randn(2, width + 1, height + 1, 1, 1, device=device)
    xs = torch.linspace(0, 1, scale + 1)[:-1, None].to(device)
    ys = torch.linspace(0, 1, scale + 1)[None, :-1].to(device)
    wx = 1 - interp(xs)
    wy = 1 - interp(ys)
    dots = 0
    dots += wx * wy * (gx[:-1, :-1] * xs + gy[:-1, :-1] * ys)
    dots += (1 - wx) * wy * (-gx[1:, :-1] * (1 - xs) + gy[1:, :-1] * ys)
    dots += wx * (1 - wy) * (gx[:-1, 1:] * xs - gy[:-1, 1:] * (1 - ys))
    dots += (1 - wx) * (1 - wy) * (-gx[1:, 1:] * (1 - xs) - gy[1:, 1:] * (1 - ys))
    return dots.permute(0, 2, 1, 3).contiguous().view(width * scale, height * scale)

def perlin_ms(octaves, width, height, grayscale, device=device):
    out_array = [0.5] if grayscale else [0.5, 0.5, 0.5]
    # out_array = [0.0] if grayscale else [0.0, 0.0, 0.0]
    for i in range(1 if grayscale else 3):
        scale = 2 ** len(octaves)
        oct_width = width
        oct_height = height
        for oct in octaves:
            p = perlin(oct_width, oct_height, scale, device)
            out_array[i] += p * oct
            scale //= 2
            oct_width *= 2
            oct_height *= 2
    return torch.cat(out_array)

def create_perlin_noise(octaves=[1, 1, 1, 1], width=2, height=2, grayscale=True):
    out = perlin_ms(octaves, width, height, grayscale)
    if grayscale:
        out = TF.resize(size=(side_y, side_x), img=out.unsqueeze(0))
        out = TF.to_pil_image(out.clamp(0, 1)).convert('RGB')
    else:
        out = out.reshape(-1, 3, out.shape[0]//3, out.shape[1])
        out = TF.resize(size=(side_y, side_x), img=out)
        out = TF.to_pil_image(out.clamp(0, 1).squeeze())

    out = ImageOps.autocontrast(out)
    return out

def regen_perlin():
    if perlin_mode == 'color':
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, False)
    elif perlin_mode == 'gray':
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, True)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
    else:
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)

    init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device).unsqueeze(0).mul(2).sub(1)
    del init2
    return init.expand(batch_size, -1, -1, -1)

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

def read_image_workaround(path):
    """OpenCV reads images as BGR, Pillow saves them as RGB. Work around
    this incompatibility to avoid colour inversions."""
    im_tmp = cv2.imread(path)
    return cv2.cvtColor(im_tmp, cv2.COLOR_BGR2RGB)

def parse_prompt(prompt):
    if prompt.startswith('http://') or prompt.startswith('https://'):
        vals = prompt.rsplit(':', 2)
        vals = [vals[0] + ':' + vals[1], *vals[2:]]
    else:
        vals = prompt.rsplit(':', 1)
    vals = vals + ['', '1'][len(vals):]
    return vals[0], float(vals[1])

def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))

def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()

def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.reshape([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.reshape([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, skip_augs=False):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.skip_augs = skip_augs
        self.augs = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomAffine(degrees=15, translate=(0.1, 0.1)),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomPerspective(distortion_scale=0.4, p=0.7),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomGrayscale(p=0.15),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            # T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        ])

    def forward(self, input):
        input = T.Pad(input.shape[2]//4, fill=0)(input)
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)

        cutouts = []
        for ch in range(self.cutn):
            if ch > self.cutn - self.cutn//4:
                cutout = input.clone()
            else:
                size = int(max_size * torch.zeros(1,).normal_(mean=.8, std=.3).clip(float(self.cut_size/max_size), 1.))
                offsetx = torch.randint(0, abs(sideX - size + 1), ())
                offsety = torch.randint(0, abs(sideY - size + 1), ())
                cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]

            if not self.skip_augs:
                cutout = self.augs(cutout)
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
            del cutout

        cutouts = torch.cat(cutouts, dim=0)
        return cutouts

cutout_debug = False
padargs = {}

class MakeCutoutsDango(nn.Module):
    def __init__(self, cut_size,
                 Overview=4, 
                 InnerCrop = 0, IC_Size_Pow=0.5, IC_Grey_P = 0.2
                 ):
        super().__init__()
        self.cut_size = cut_size
        self.Overview = Overview
        self.InnerCrop = InnerCrop
        self.IC_Size_Pow = IC_Size_Pow
        self.IC_Grey_P = IC_Grey_P
        if args.animation_mode == 'None':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.5),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=10, translate=(0.05, 0.05),  interpolation = T.InterpolationMode.BILINEAR),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.1),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
          ])
        elif args.animation_mode == 'Video Input':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.5),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=15, translate=(0.1, 0.1)),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomPerspective(distortion_scale=0.4, p=0.7),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.15),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              # T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
          ])
        elif  args.animation_mode == '2D' or args.animation_mode == '3D':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.4),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=10, translate=(0.05, 0.05),  interpolation = T.InterpolationMode.BILINEAR),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.1),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.3),
          ])
          

    def forward(self, input):
        cutouts = []
        gray = T.Grayscale(3)
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        l_size = max(sideX, sideY)
        output_shape = [1,3,self.cut_size,self.cut_size] 
        output_shape_2 = [1,3,self.cut_size+2,self.cut_size+2]
        pad_input = F.pad(input,((sideY-max_size)//2,(sideY-max_size)//2,(sideX-max_size)//2,(sideX-max_size)//2), **padargs)
        cutout = resize(pad_input, out_shape=output_shape)

        if self.Overview>0:
            if self.Overview<=4:
                if self.Overview>=1:
                    cutouts.append(cutout)
                if self.Overview>=2:
                    cutouts.append(gray(cutout))
                if self.Overview>=3:
                    cutouts.append(TF.hflip(cutout))
                if self.Overview==4:
                    cutouts.append(gray(TF.hflip(cutout)))
            else:
                cutout = resize(pad_input, out_shape=output_shape)
                for _ in range(self.Overview):
                    cutouts.append(cutout)

            if cutout_debug:
                if is_colab:
                    TF.to_pil_image(cutouts[0].clamp(0, 1).squeeze(0)).save("/content/cutout_overview0.jpg",quality=99)
                else:
                    TF.to_pil_image(cutouts[0].clamp(0, 1).squeeze(0)).save("cutout_overview0.jpg",quality=99)

                              
        if self.InnerCrop >0:
            for i in range(self.InnerCrop):
                size = int(torch.rand([])**self.IC_Size_Pow * (max_size - min_size) + min_size)
                offsetx = torch.randint(0, sideX - size + 1, ())
                offsety = torch.randint(0, sideY - size + 1, ())
                cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
                if i <= int(self.IC_Grey_P * self.InnerCrop):
                    cutout = gray(cutout)
                cutout = resize(cutout, out_shape=output_shape)
                cutouts.append(cutout)
            if cutout_debug:
                if is_colab:
                    TF.to_pil_image(cutouts[-1].clamp(0, 1).squeeze(0)).save("/content/cutout_InnerCrop.jpg",quality=99)
                else:
                    TF.to_pil_image(cutouts[-1].clamp(0, 1).squeeze(0)).save("cutout_InnerCrop.jpg",quality=99)
        cutouts = torch.cat(cutouts)
        if skip_augs is not True: cutouts=self.augs(cutouts)
        return cutouts

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)     

def tv_loss(input):
    """L2 total variation loss, as in Mahendran et al."""
    input = F.pad(input, (0, 1, 0, 1), 'replicate')
    x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
    y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
    return (x_diff**2 + y_diff**2).mean([1, 2, 3])


def range_loss(input):
    return (input - input.clamp(-1, 1)).pow(2).mean([1, 2, 3])

stop_on_next_loop = False  # Make sure GPU memory doesn't get corrupted from cancelling the run mid-way through, allow a full frame to complete
TRANSLATION_SCALE = 1.0/200.0

def do_3d_step(img_filepath, frame_num, midas_model, midas_transform):
  if args.key_frames:
    translation_x = args.translation_x_series[frame_num]
    translation_y = args.translation_y_series[frame_num]
    translation_z = args.translation_z_series[frame_num]
    rotation_3d_x = args.rotation_3d_x_series[frame_num]
    rotation_3d_y = args.rotation_3d_y_series[frame_num]
    rotation_3d_z = args.rotation_3d_z_series[frame_num]
    print(
        f'translation_x: {translation_x}',
        f'translation_y: {translation_y}',
        f'translation_z: {translation_z}',
        f'rotation_3d_x: {rotation_3d_x}',
        f'rotation_3d_y: {rotation_3d_y}',
        f'rotation_3d_z: {rotation_3d_z}',
    )

  translate_xyz = [-translation_x*TRANSLATION_SCALE, translation_y*TRANSLATION_SCALE, -translation_z*TRANSLATION_SCALE]
  rotate_xyz_degrees = [rotation_3d_x, rotation_3d_y, rotation_3d_z]
  print('translation:',translate_xyz)
  print('rotation:',rotate_xyz_degrees)
  rotate_xyz = [math.radians(rotate_xyz_degrees[0]), math.radians(rotate_xyz_degrees[1]), math.radians(rotate_xyz_degrees[2])]
  rot_mat = p3dT.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
  print("rot_mat: " + str(rot_mat))
  next_step_pil = dxf.transform_image_3d(img_filepath, midas_model, midas_transform, DEVICE,
                                          rot_mat, translate_xyz, math.floor(args.near_plane_series[frame_num]), math.floor(args.far_plane_series[frame_num]),
                                          math.floor(args.fov_series[frame_num]), padding_mode=args.padding_mode,
                                          sampling_mode=args.sampling_mode, midas_weight=args.midas_weight_series[frame_num])
  return next_step_pil

def soft_limit(x):
  #zippy soft limiter
  #value from -n to n, always return a value -1<x<1
  #soft_limiter_knee set in params = 0.97 # where does compression start?
  soft_sign = x/abs(x)
  soft_overage = ((abs(x)-soft_limiter_knee)+(abs(abs(x)-soft_limiter_knee)))/2
  soft_base = abs(x)-soft_overage
  soft_limited_x = soft_base + torch.tanh(soft_overage/(1-soft_limiter_knee))*(1-soft_limiter_knee)
  return soft_limited_x*soft_sign

def symmetry_transformation_fn(x):
  if args.use_horizontal_symmetry:
    [n, c, h, w] = x.size()
    x = torch.concat((x[:, :, :, :w//2], torch.flip(x[:, :, :, :w//2], [-1])), -1)
    print("horizontal symmetry applied")
  if args.use_vertical_symmetry:
    [n, c, h, w] = x.size()
    x = torch.concat((x[:, :, :h//2, :], torch.flip(x[:, :, :h//2, :], [-2])), -2)
    print("vertical symmetry applied")
  return x

def determine_zoom_scale(box_width, box_height, image_width, image_height, num_iterations):
  if image_width/box_width > image_height/box_height:
    small_number = box_width
    large_number = image_width
  else:
      small_number = box_height
      large_number = image_height 
  percent_increase = (large_number / small_number) ** (1/num_iterations)
  return percent_increase

def calculate_box_center_pixel(box_coordinates):
  box_center_pixel_horizontal = (box_coordinates[0] + box_coordinates[1])//2
  box_center_pixel_vertical = (box_coordinates[2] + box_coordinates[3])//2
  return box_center_pixel_horizontal, box_center_pixel_vertical

def request_caption_image(init_image):
  antarctic_prompt_request_directory = '/content/drive/MyDrive/AI/antarctic_prompt/request/'
  #write init_image to antarctic_prompt_request_directory as a png
  cv_init_image = cv2.imread(init_image)
  print('cv_init_image', cv_init_image)
  cv2.imwrite(antarctic_prompt_request_directory+"/init_image.png", cv_init_image)
 #it doesn't like this, maybe try making it as similar to ODs as possible, maybe check the types of each and compare, maybe need to do that other cv thing before doing this
  antarctic_prompts = []
  antarctic_prompt_start_time=time.time()
  while len(antarctic_prompts) == 0:
    for file in os.listdir(antarctic_prompt_request_directory):
      if file.endswith('.txt'):
        #open the file and put the lines into a list called antarctic_prompts
        with open(antarctic_prompt_request_directory +"/"+ file, 'r') as f:
          antarctic_prompts = f.readlines() 
        os.remove(antarctic_prompt_request_directory +"/antarctic_prompts.txt")      
      else:
        time.sleep(1)
  antarctic_prompt_end_time=time.time()
  antarctic_prompt_duration_time= antarctic_prompt_end_time-antarctic_prompt_start_time
  antarctic_prompt_durations.append(antarctic_prompt_duration_time)
  stats['antarctic_prompt_durations'] = antarctic_prompt_durations
  average_antarctic_prompt_duration = sum(antarctic_prompt_durations) / len(antarctic_prompt_durations)
  stats['average_antarctic_prompt_duration'] = average_antarctic_prompt_duration                            
  with open(f"{batchFolder}/{batch_name}({batchNum})_stats.txt", "w+") as ant: 
    json.dump(stats, ant, ensure_ascii=False, indent=4)
  return antarctic_prompts       

#todo text file should be deleted after it gets used!!!!!!!!!!!!!
def do_run():
  global stats
  global widget
  if os.path.isfile(f"{batchFolder}/{batch_name}({batchNum})_stats.txt"):
    with open(f'{batchFolder}/{batch_name}({batchNum})_stats.txt') as file:
      stats = json.load(file)
  seed = args.seed
  print(range(args.start_frame, args.max_frames))
  
  updated_antarctic_text_prompts_series = args.prompts_series
  #list_of_rot_mat_centers = (1*args.width_height[1]//2, 1*args.width_height[0]//2) * args.max_frames
  list_of_rot_mat_centers = [() for _ in range(args.max_frames + 30)]
  if (args.animation_mode == "3D") and (args.midas_weight > 0.0):
      midas_model, midas_transform, midas_net_w, midas_net_h, midas_resize_mode, midas_normalization = init_midas_depth_model(args.midas_depth_model)
  frame_durations = []
  object_detection_durations=[]
  antarctic_prompt_durations=[]
  average_object_detection_duration = 0
  average_antarctic_prompt_duration = 0
  average_frame_duration = 0
  previous_frame_time = round(time.time() * 1000)
  print('args.translation_x_series', args.translation_x_series)
  print('args.translation_y_series', args.translation_y_series)
  # translation_x_series.update(box_zoom_variable_updates['translation_x_series'])
  # translation_y_series.update(box_zoom_variable_updates['translation_y_series'])
  # zoom_series.update(box_zoom_variable_updates['zoom_series'])
  list_of_rot_mat_centers = box_zoom_variable_updates['rot_mat_center_series'].copy()
  #we should be able to use update for the dictionaries, but i think rot_mat is a list of tuples so it may need to be different
  for frame_num in range(args.start_frame, args.max_frames):

      if stop_on_next_loop:
        break

      current_frame_time = round(time.time() * 1000)
      frame_durations.append(current_frame_time-previous_frame_time)
      stats['frame_durations'] = frame_durations                            
      previous_frame_time = current_frame_time
      average_frame_duration = sum(frame_durations) / len(frame_durations)
      stats['average_frame_duration'] = average_frame_duration                            
      with open(f"{batchFolder}/{batch_name}({batchNum})_stats.txt", "w+") as f: 
        json.dump(stats, f, ensure_ascii=False, indent=4)
      
      display.clear_output(wait=True)

      # Print Frame progress if animation mode is on
      if args.animation_mode != "None":
        batchBar = tqdm(range(args.max_frames), desc ="Frames")
        batchBar.n = frame_num
        batchBar.refresh()

      
      # Inits if not video frames
      if args.animation_mode != "Video Input":
        if args.init_image in ['','none', 'None', 'NONE']:
          init_image = None
        else:
          init_image = args.init_image
        init_scale = args.init_scale
        #skip_steps = args.skip_steps + (frame_num * 10) #tj maybe try making a notification to see if we keep coming here
        skip_steps = 0
      if args.animation_mode == "2D":
        if args.key_frames:
          angle = args.angle_series[frame_num]
          zoom = args.zoom_series[frame_num]
          object_detection_zoom = args.object_detection_zoom
          translation_x = args.translation_x_series[frame_num]
          translation_y = args.translation_y_series[frame_num]
          print(
              f'angle: {angle}',
              f'zoom: {zoom}',
              f'object_detection_zoom: {object_detection_zoom}',
              f'translation_x: {translation_x}',
              f'translation_y: {translation_y}',
              f'translation_x_series: {translation_x_series}',
              f'translation_y_series: {translation_y_series}',
              f'zoom_series: {zoom_series}',
              f'new_frames_scale_series: {new_frames_scale_series}',
              f'new_frames_skip_steps_series: {new_frames_skip_steps_series}',
          )
          print('next_frame', frame_num)
          print('box_zoom_variable_updates',box_zoom_variable_updates)
          #todo should be able to plug in the stuff in this function to make the zoom happen
          if os.path.exists('./prevFrame.png'):
            print('it exists', frame_num)
            def encode_image(filepath):
                with open('./prevFrame.png', 'rb') as f:
                    image_bytes = f.read()
                encoded = str(base64.b64encode(image_bytes), 'utf-8')
                return "data:image/jpg;base64,"+encoded
            guided_zoom_widget.image = encode_image('./prevFrame.png')
            #widget.bboxes = []
        
        if frame_num > 0:
          seed += 1
          if resume_run and frame_num == start_frame:
            img_0 = cv2.imread(batchFolder+f"/{batch_name}({batchNum})_{start_frame-1:04}.png")
          else:
            img_0 = cv2.imread('prevFrame.png')
          
          #center = list_of_rot_mat_centers[frame_num]
          center = (1*img_0.shape[1]//2, 1*img_0.shape[0]//2)
          if len(list_of_rot_mat_centers) > frame_num:
            center = list_of_rot_mat_centers[frame_num]
          if frame_num in object_detection_zoom:
            OD_box_coordinates_string = ''
            OD_request_folder = '/content/drive/MyDrive/AI/Object_Detection/request'
            #save img_0 to OD_request_folder
            cv2.imwrite(OD_request_folder+f"/{batch_name}({batchNum})_{frame_num:04}.png", img_0)
            object_detection_start_time= time.time()
            while OD_box_coordinates_string == '':             
              for file in os.listdir(OD_request_folder):
                if file.endswith('.txt'):
                  #get box_coordinates from file
                  with open(OD_request_folder+'/'+file, 'r') as f:
                    OD_box_coordinates_string = f.read()
                  os.remove(OD_request_folder+'/'+file)
                else:
                  time.sleep(1)
            #TODO COPY THIS STUFF TO BOX ZOOM CELL
            object_detection_end_time= time.time()
            object_detection_duration_time= object_detection_end_time-object_detection_start_time
            object_detection_durations.append(object_detection_duration_time)
            stats['object_detection_durations'] = object_detection_durations
            average_object_detection_duration = sum(object_detection_durations) / len(object_detection_durations)
            stats['average_object_detection_duration'] = average_object_detection_duration                            
            with open(f"{batchFolder}/{batch_name}({batchNum})_stats.txt", "w+") as obj: 
              json.dump(stats, obj, ensure_ascii=False, indent=4)
            

            num_iterations = 30 #todo this will eventually be a setting
            OD_box_coordinates_list = OD_box_coordinates_string.split(',')
            box_coordinates = []
            for cord in OD_box_coordinates_list:
              box_coordinates.append(int(float(cord)))
            box_width = abs(box_coordinates[0] - box_coordinates[1]) * (args.width_height[0]/256)
            box_height = abs(box_coordinates[2] - box_coordinates[3]) * (args.width_height[1]/256)
            zoom_scale = determine_zoom_scale(box_width, box_height, args.width_height[0], args.width_height[1], num_iterations)
            box_center_x, box_center_y = calculate_box_center_pixel(box_coordinates)
            x_translate_per_iteration = int(((args.width_height[0]//2) - box_center_x) / num_iterations)
            y_translate_per_iteration = int(((args.width_height[1]//2) - box_center_y) / num_iterations)
            for frame in range(frame_num, frame_num+num_iterations):
              iteration_number = frame + 1 - frame_num
              translation_x_series[frame] = x_translate_per_iteration
              translation_y_series[frame] = y_translate_per_iteration
              list_of_rot_mat_centers[frame] = (box_center_x+(iteration_number*x_translate_per_iteration), box_center_y+(iteration_number*y_translate_per_iteration))
              zoom_series[frame] = zoom_scale
            #check to see if center is a tuple
            center = list_of_rot_mat_centers[frame_num]
            print('centertype', type(center))
            translation_x = args.translation_x_series[frame_num]
            translation_y = args.translation_y_series[frame_num]
            zoom = args.zoom_series[frame_num]
            #todo eventually make the number of frames that it takes to be zoomed in be based on the future zoom series(for now it is 30)
          
          trans_mat = np.float32(
              [[1, 0, translation_x],
              [0, 1, translation_y]]
          )
          if len(center) == 0:
            center = (1*img_0.shape[1]//2, 1*img_0.shape[0]//2)
          rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )
          trans_mat = np.vstack([trans_mat, [0,0,1]])
          rot_mat = np.vstack([rot_mat, [0,0,1]])
          transformation_matrix = np.matmul(rot_mat, trans_mat)
          img_0 = cv2.warpPerspective(
              img_0,
              transformation_matrix,
              (img_0.shape[1], img_0.shape[0]),
              borderMode=cv2.BORDER_WRAP
          )

          cv2.imwrite('prevFrameScaled.png', img_0)
          init_image = 'prevFrameScaled.png'
          init_scale = math.floor(args.new_frames_scale_series[frame_num])
          # skip_steps = args.calc_frames_skip_steps#tj this is overriding my skip steps i think
          # #skip_steps = args.skip_steps + (frame_num * 10)
          # if len(args.calc_frames_skip_steps_series) > 0:
          #   # print('args.calc_frames_skip_steps_series', args.calc_frames_skip_steps_series)
          #   # print('args.new_cutn_batches_series', args.new_cutn_batches_series)
          #   # print('args.new_clip_guidance_scale_series', args.new_clip_guidance_scale_series)
          #   # print('args.new_range_scale_series',args.new_range_scale_series)
          skip_steps = args.calc_frames_skip_steps_series[frame_num]
      if args.animation_mode == "3D":
        if frame_num > 0:
          seed += 1    
          if resume_run and frame_num == start_frame:
            img_filepath = batchFolder+f"/{batch_name}({batchNum})_{start_frame-1:04}.png"
            if turbo_mode and frame_num > turbo_preroll:
              shutil.copyfile(img_filepath, 'oldFrameScaled.png')
          else:
            img_filepath = '/content/prevFrame.png' if is_colab else 'prevFrame.png'

          next_step_pil = do_3d_step(img_filepath, frame_num, midas_model, midas_transform)
          next_step_pil.save('prevFrameScaled.png')

          ### Turbo mode - skip some diffusions, use 3d morph for clarity and to save time
          if turbo_mode:
            if frame_num == turbo_preroll: #start tracking oldframe
              next_step_pil.save('oldFrameScaled.png')#stash for later blending          
            elif frame_num > turbo_preroll:
              #set up 2 warped image sequences, old & new, to blend toward new diff image
              old_frame = do_3d_step('oldFrameScaled.png', frame_num, midas_model, midas_transform)
              old_frame.save('oldFrameScaled.png')
              if frame_num % int(turbo_steps) != 0: 
                print('turbo skip this frame: skipping clip diffusion steps')
                filename = f'{args.batch_name}({args.batchNum})_{frame_num:04}.png'
                blend_factor = ((frame_num % int(turbo_steps))+1)/int(turbo_steps)
                print('turbo skip this frame: skipping clip diffusion steps and saving blended frame')
                newWarpedImg = cv2.imread('prevFrameScaled.png')#this is already updated..
                oldWarpedImg = cv2.imread('oldFrameScaled.png')
                blendedImage = cv2.addWeighted(newWarpedImg, blend_factor, oldWarpedImg,1-blend_factor, 0.0)
                cv2.imwrite(f'{batchFolder}/{filename}',blendedImage)
                next_step_pil.save(f'{img_filepath}') # save it also as prev_frame to feed next iteration
                if vr_mode:
                  generate_eye_views(TRANSLATION_SCALE,batchFolder,filename,frame_num,midas_model, midas_transform)
                continue
              else:
                #if not a skip frame, will run diffusion and need to blend.
                oldWarpedImg = cv2.imread('prevFrameScaled.png')
                cv2.imwrite(f'oldFrameScaled.png',oldWarpedImg)#swap in for blending later 
                print('clip/diff this frame - generate clip diff image')
          init_image = 'prevFrameScaled.png'
          init_scale = math.floor(args.new_frames_scale_series[frame_num])
          skip_steps = args.calc_frames_skip_steps
      if  args.animation_mode == "Video Input":
        if not video_init_seed_continuity:
          seed += 1
        init_image = f'{videoFramesFolder}/{frame_num+1:04}.jpg'
        init_scale = math.floor(args.new_frames_scale_series[frame_num])
        skip_steps = args.calc_frames_skip_steps
      loss_values = []  
      if seed is not None:
          np.random.seed(seed)
          random.seed(seed)
          torch.manual_seed(seed)
          torch.cuda.manual_seed_all(seed)
          torch.backends.cudnn.deterministic = True  
      target_embeds, weights = [], []
      if antarctic_prompt_frames:
        should_use_antarctic_prompt = False
        #if "x" is in antarctic_prompt_frames:
        if "x" in antarctic_prompt_frames and frame_num != 0:
          #number_in_antarctic_prompt_frames is antarctic_prompt_frames without the x
          number_in_antarctic_prompt_frames = antarctic_prompt_frames.replace("x","")
          if frame_num % int(number_in_antarctic_prompt_frames) == 0:
            should_use_antarctic_prompt = True
        elif str(frame_num) in antarctic_prompt_frames.split(","):
          should_use_antarctic_prompt = True
        if should_use_antarctic_prompt:
          global updated_antarctic_text_prompts
          #new_antarctic_prompts = caption_image(init_image, antarctic_args, antarctic_net, antarctic_preprocess)[:antarctic_number_of_captions]
          #trying new request method
          new_antarctic_prompts = request_caption_image(init_image)#, antarctic_args, antarctic_net, antarctic_preprocess)[:antarctic_number_of_captions]
          #every string in new_antarctic_prompts has antarctic_pre_string and antarctic_post_string prepended and appended to it
          for i in range(len(new_antarctic_prompts)):
            new_antarctic_prompts[i] = antarctic_pre_string + new_antarctic_prompts[i] + antarctic_post_string          
          updated_antarctic_text_prompts[frame_num] = new_antarctic_prompts
          updated_antarctic_text_prompts[frame_num].extend(antarctic_additional_prompts)
          updated_antarctic_text_prompts = dict(sorted(updated_antarctic_text_prompts.items(), key=lambda x: int(x[0])))
          #json dump dict to file
          with open(f'{batchFolder}/antarctic_text_prompts.txt', 'w') as fp:
            json.dump(updated_antarctic_text_prompts, fp, ensure_ascii=False, indent=4)
          updated_antarctic_text_prompts_series = split_prompts(updated_antarctic_text_prompts)
        if updated_antarctic_text_prompts_series is not None and frame_num >= len(updated_antarctic_text_prompts_series):
          frame_prompt = updated_antarctic_text_prompts_series[-1]
        elif updated_antarctic_text_prompts_series is not None:
          frame_prompt = updated_antarctic_text_prompts_series[frame_num]
        else:
          frame_prompt = []        
      else:
        if args.prompts_series is not None and frame_num >= len(args.prompts_series):
          frame_prompt = args.prompts_series[-1]
        elif args.prompts_series is not None:
          frame_prompt = args.prompts_series[frame_num]
        else:
          frame_prompt = []
      print(args.image_prompts_series)
      if args.image_prompts_series is not None and frame_num >= len(args.image_prompts_series):
        image_prompt = args.image_prompts_series[-1]
      elif args.image_prompts_series is not None:
        image_prompt = args.image_prompts_series[frame_num]
      else:
        image_prompt = []

      print(f'Frame {frame_num} Prompt: {frame_prompt}')

      model_stats = []
      for clip_model in clip_models:
            cutn = 16
            model_stat = {"clip_model":None,"target_embeds":[],"make_cutouts":None,"weights":[]}
            model_stat["clip_model"] = clip_model
            for prompt in frame_prompt:
                txt, weight = parse_prompt(prompt)
                txt = clip_model.encode_text(clip.tokenize(prompt).to(device)).float()
                
                if args.fuzzy_prompt:
                    for i in range(25):
                        model_stat["target_embeds"].append((txt + torch.randn(txt.shape).cuda() * args.rand_mag).clamp(0,1))
                        model_stat["weights"].append(weight)
                else:
                    model_stat["target_embeds"].append(txt)
                    model_stat["weights"].append(weight)
        
            if image_prompt:
              model_stat["make_cutouts"] = MakeCutouts(clip_model.visual.input_resolution, cutn, skip_augs=skip_augs) 
              for prompt in image_prompt:
                  path, weight = parse_prompt(prompt)
                  img = Image.open(fetch(path)).convert('RGB')
                  img = TF.resize(img, min(side_x, side_y, *img.size), T.InterpolationMode.LANCZOS)
                  batch = model_stat["make_cutouts"](TF.to_tensor(img).to(device).unsqueeze(0).mul(2).sub(1))
                  embed = clip_model.encode_image(normalize(batch)).float()
                  if fuzzy_prompt:
                      for i in range(25):
                          model_stat["target_embeds"].append((embed + torch.randn(embed.shape).cuda() * rand_mag).clamp(0,1))
                          weights.extend([weight / cutn] * cutn)
                  else:
                      model_stat["target_embeds"].append(embed)
                      model_stat["weights"].extend([weight / cutn] * cutn)
        
            model_stat["target_embeds"] = torch.cat(model_stat["target_embeds"])
            model_stat["weights"] = torch.tensor(model_stat["weights"], device=device)
            if model_stat["weights"].sum().abs() < 1e-3:
                raise RuntimeError('The weights must not sum to 0.')
            model_stat["weights"] /= model_stat["weights"].sum().abs()
            model_stats.append(model_stat)
  
      init = None
      if init_image is not None:
          init = Image.open(fetch(init_image)).convert('RGB')
          init = init.resize((args.side_x, args.side_y), Image.LANCZOS)
          init = TF.to_tensor(init).to(device).unsqueeze(0).mul(2).sub(1)
      
      if args.perlin_init:
          if args.perlin_mode == 'color':
              init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
              init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, False)
          elif args.perlin_mode == 'gray':
            init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, True)
            init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
          else:
            init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
            init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
          # init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device)
          init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device).unsqueeze(0).mul(2).sub(1)
          del init2
  
      cur_t = None
  
      def cond_fn(x, t, y=None):
          with torch.enable_grad():
              x_is_NaN = False
              x = x.detach().requires_grad_()
              n = x.shape[0]
              if use_secondary_model is True:
                alpha = torch.tensor(diffusion.sqrt_alphas_cumprod[cur_t], device=device, dtype=torch.float32)
                sigma = torch.tensor(diffusion.sqrt_one_minus_alphas_cumprod[cur_t], device=device, dtype=torch.float32)
                cosine_t = alpha_sigma_to_t(alpha, sigma)
                out = secondary_model(x, cosine_t[None].repeat([n])).pred
                fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
                x_in = out * fac + x * (1 - fac)
                x_in_grad = torch.zeros_like(x_in)
              else:
                my_t = torch.ones([n], device=device, dtype=torch.long) * cur_t
                out = diffusion.p_mean_variance(model, x, my_t, clip_denoised=False, model_kwargs={'y': y})
                fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
                x_in = out['pred_xstart'] * fac + x * (1 - fac)
                x_in_grad = torch.zeros_like(x_in)
              for model_stat in model_stats:
                #for i in range(args.cutn_batches): tj - this is how it was, my new line is the next line
                for i in range(math.floor(args.new_cutn_batches_series[frame_num])):                  
                    t_int = int(t.item())+1 #errors on last step without +1, need to find source
                    #when using SLIP Base model the dimensions need to be hard coded to avoid AttributeError: 'VisionTransformer' object has no attribute 'input_resolution'
                    try:
                        input_resolution=model_stat["clip_model"].visual.input_resolution
                    except:
                        input_resolution=224

                    cuts = MakeCutoutsDango(input_resolution,
                            Overview= args.cut_overview[1000-t_int], 
                            InnerCrop = args.cut_innercut[1000-t_int], IC_Size_Pow=args.new_cut_ic_pow_series[frame_num], IC_Grey_P = args.cut_icgray_p[1000-t_int]
                            )
                    clip_in = normalize(cuts(x_in.add(1).div(2)))
                    image_embeds = model_stat["clip_model"].encode_image(clip_in).float()
                    dists = spherical_dist_loss(image_embeds.unsqueeze(1), model_stat["target_embeds"].unsqueeze(0))
                    dists = dists.view([args.cut_overview[1000-t_int]+args.cut_innercut[1000-t_int], n, -1])
                    losses = dists.mul(model_stat["weights"]).sum(2).mean(0)
                    loss_values.append(losses.sum().item()) # log loss, probably shouldn't do per cutn_batch
                    x_in_grad += torch.autograd.grad(losses.sum() * math.floor(args.new_clip_guidance_scale_series[frame_num]), x_in)[0] / math.floor(args.new_cutn_batches_series[frame_num])
              tv_losses = tv_loss(x_in)
              if use_secondary_model is True:
                range_losses = range_loss(out)
              else:
                range_losses = range_loss(out['pred_xstart'])
              sat_losses = torch.abs(x_in - x_in.clamp(min=-1,max=1)).mean()
              loss = tv_losses.sum() * math.floor(args.new_tv_scale_series[frame_num]) + range_losses.sum() * math.floor(args.new_range_scale_series[frame_num]) + sat_losses.sum() * math.floor(args.new_sat_scale_series[frame_num])
              if init is not None and args.init_scale:
                  init_losses = lpips_model(x_in, init)
                  loss = loss + init_losses.sum() * args.init_scale
              x_in_grad += torch.autograd.grad(loss, x_in)[0]
              if torch.isnan(x_in_grad).any()==False:
                  grad = -torch.autograd.grad(x_in, x, x_in_grad)[0]
              else:
                # print("NaN'd")
                x_is_NaN = True
                grad = torch.zeros_like(x)
          if args.clamp_grad and x_is_NaN == False:
              magnitude = grad.square().mean().sqrt()
              return grad * magnitude.clamp(max=args.new_clamp_max_series[frame_num]) / magnitude  #min=-0.02, min=-clamp_max, 
          return grad
  
      if args.diffusion_sampling_mode == 'ddim':
          sample_fn = diffusion.ddim_sample_loop_progressive
      else:
          sample_fn = diffusion.plms_sample_loop_progressive


      image_display = Output()
      for i in range(args.n_batches):
          if args.animation_mode == 'None':
            display.clear_output(wait=True)
            batchBar = tqdm(range(args.n_batches), desc ="Batches")
            batchBar.n = i
            batchBar.refresh()
          print('')
          display.display(image_display)
          gc.collect()
          torch.cuda.empty_cache()
          cur_t = diffusion.num_timesteps - skip_steps - 1
          #tj cur_t = math.floor(args.anim_steps_series[frame_num] - skip_steps - 1)
          
          total_steps = cur_t

          if perlin_init:
              init = regen_perlin()

          if args.diffusion_sampling_mode == 'ddim':
              samples = sample_fn(
                  model,
                  (batch_size, 3, args.side_y, args.side_x),
                  clip_denoised=clip_denoised,
                  model_kwargs={},
                  cond_fn=cond_fn,
                  progress=True,
                  skip_timesteps=skip_steps,
                  init_image=init,
                  randomize_class=randomize_class,
                  eta=args.new_eta_series[frame_num],
                  transformation_fn=symmetry_transformation_fn,
                  transformation_percent=args.transformation_percent
              )
          else:
              samples = sample_fn(
                  model,
                  (batch_size, 3, args.side_y, args.side_x),
                  clip_denoised=clip_denoised,
                  model_kwargs={},
                  cond_fn=cond_fn,
                  progress=True,
                  skip_timesteps=skip_steps,
                  init_image=init,
                  randomize_class=randomize_class,
                  order=2,
              )
          print('used_series: ', args.used_series)
          print('frame_durations: ',frame_durations)
          print('average_frame_duration: ' + str(average_frame_duration))
          print('args.batch_name: ', args.batch_name)
          for series in args.used_series:
            print('this series', series)
            if hasattr(args, series+"_series"):
              print("args."+series+"_series: "+ str(getattr(args, series+"_series")[frame_num]))
          if len(animations_with_keyframes_higher_than_max) > 0:
            print('WARNING: animations_with_keyframes_higher_than_max: ', animations_with_keyframes_higher_than_max)
          #if the largest prompt key in image_prompts_series is greater than max keyframes, then tell the user this
          if int(args.prompts_series.keys()[-1]) > int(args.max_frames):
            print('WARNING: there is a prompt higher than the max keyframes')

          # with run_display:
          # display.clear_output(wait=True)
          for j, sample in enumerate(samples):    
            cur_t -= 1
            intermediateStep = False
            if args.steps_per_checkpoint is not None:
                if j % steps_per_checkpoint == 0 and j > 0:
                  intermediateStep = True
            elif j in args.intermediate_saves:
              intermediateStep = True
            with image_display:
              if j % args.display_rate == 0 or cur_t == -1 or intermediateStep == True:
                  for k, image in enumerate(sample['pred_xstart']):
                      # tqdm.write(f'Batch {i}, step {j}, output {k}:')
                      current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
                      percent = math.ceil(j/total_steps*100)
                      if args.n_batches > 0:
                        #if intermediates are saved to the subfolder, don't append a step or percentage to the name
                        if cur_t == -1 and args.intermediates_in_subfolder is True:
                          save_num = f'{frame_num:04}' if animation_mode != "None" else i
                          filename = f'{args.batch_name}({args.batchNum})_{save_num}.png'
                        else:
                          #If we're working with percentages, append it
                          if args.steps_per_checkpoint is not None:
                            filename = f'{args.batch_name}({args.batchNum})_{i:04}-{percent:02}%.png'
                          # Or else, iIf we're working with specific steps, append those
                          else:
                            filename = f'{args.batch_name}({args.batchNum})_{i:04}-{j:03}.png'
                      #Optional Soft Limiter
                      if args.soft_limiter_on == True:
                          #zippy's soft limiter
                          image=(soft_limit(image)+1)/2
                          image = TF.to_pil_image(image)
                      else:
                          #default clamping behavior
                          #clamp values to between 0 and 1
                          image = TF.to_pil_image(image.add(1).div(2).clamp(0, 1))
                      if j % args.display_rate == 0 or cur_t == -1:
                        image.save('progress.png')
                        display.clear_output(wait=True)
                        display.display(display.Image('progress.png'))
                      if args.steps_per_checkpoint is not None:
                        if j % args.steps_per_checkpoint == 0 and j > 0:
                          if args.intermediates_in_subfolder is True:
                            image.save(f'{partialFolder}/{filename}')
                          else:
                            image.save(f'{batchFolder}/{filename}')
                      else:
                        if j in args.intermediate_saves:
                          if args.intermediates_in_subfolder is True:
                            image.save(f'{partialFolder}/{filename}')
                          else:
                            image.save(f'{batchFolder}/{filename}')
                      if cur_t == -1:
                        if frame_num == 0:
                          save_settings()
                        #if frame_num == args.max_frames-1:
                        save_settings()
                        if args.animation_mode != "None":
                          image.save('prevFrame.png')
                        image.save(f'{batchFolder}/{filename}')

                        if args.animation_mode == "3D":
                          # If turbo, save a blended image
                          if turbo_mode and frame_num > 0:
                            # Mix new image with prevFrameScaled
                            blend_factor = (1)/int(turbo_steps)
                            newFrame = cv2.imread('prevFrame.png') # This is already updated..
                            prev_frame_warped = cv2.imread('prevFrameScaled.png')
                            blendedImage = cv2.addWeighted(newFrame, blend_factor, prev_frame_warped, (1-blend_factor), 0.0)
                            cv2.imwrite(f'{batchFolder}/{filename}',blendedImage)
                          else:
                            image.save(f'{batchFolder}/{filename}')

                          if vr_mode:
                            generate_eye_views(TRANSLATION_SCALE, batchFolder, filename, frame_num, midas_model, midas_transform)

                        # if frame_num != args.max_frames-1:
                        #   display.clear_output()
        
          plt.plot(np.array(loss_values), 'r')

def generate_eye_views(trans_scale,batchFolder,filename,frame_num,midas_model, midas_transform):
   for i in range(2):
      theta = vr_eye_angle * (math.pi/180)
      ray_origin = math.cos(theta) * vr_ipd / 2 * (-1.0 if i==0 else 1.0)
      ray_rotation = (theta if i==0 else -theta)
      translate_xyz = [-(ray_origin)*trans_scale, 0,0]
      rotate_xyz = [0, (ray_rotation), 0]
      rot_mat = p3dT.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
      transformed_image = dxf.transform_image_3d(f'{batchFolder}/{filename}', midas_model, midas_transform, DEVICE,
                                                      rot_mat, translate_xyz, math.floor(args.near_plane_series[frame_num]), math.floor(args.far_plane_series[frame_num]),
                                                      math.floor(args.fov_series[frame_num]), padding_mode=args.padding_mode,
                                                      sampling_mode=args.sampling_mode, midas_weight=args.midas_weight_series[frame_num],spherical=True)
      eye_file_path = batchFolder+f"/frame_{frame_num:04}" + ('_l' if i==0 else '_r')+'.png'
      transformed_image.save(eye_file_path)

def save_settings(save_to_project_folder=True):
  setting_list = {
    'text_prompts': text_prompts,
    'image_prompts': image_prompts,
    'updated_antarctic_text_prompts': updated_antarctic_text_prompts,
    'overwrite_settings_file': overwrite_settings_file,
    'batch_name': batch_name,
    'absolute_frame_multiplier': absolute_frame_multiplier,
    'antarctic_prompt_frames_input': antarctic_prompt_frames_input,
    'antarctic_pre_string': antarctic_pre_string,
    'antarctic_post_string': antarctic_post_string,
    'antarctic_number_of_captions': antarctic_number_of_captions,
    'diffusion_model': diffusion_model,
    'use_secondary_model': str(use_secondary_model),
    'diffusion_steps': diffusion_steps,
    'diffusion_sampling_mode': diffusion_sampling_mode,
    'use_checkpoint': str(use_checkpoint),
    'ViTB32': str(ViTB32),
    'ViTB16': str(ViTB16),
    'ViTL14': str(ViTL14),
    'ViTL14_336px': str(ViTL14_336px),
    'RN101': str(RN101),
    'RN50': str(RN50),
    'RN50x4': str(RN50x4),
    'RN50x16': str(RN50x16),
    'RN50x64': str(RN50x64),
    'check_model_SHA': str(check_model_SHA),
    'steps': steps,
    'width': width_height[0],
    'height': width_height[1],
    'clip_guidance_scale_input': clip_guidance_scale_input,
    'tv_scale_input': tv_scale_input,
    'range_scale_input': range_scale_input,
    'sat_scale_input': sat_scale_input,
    # 'cutn': cutn,
    'cutn_batches_input': cutn_batches_input,
    'skip_augs': str(skip_augs),
    'randomize_class': randomize_class,
    'clip_denoised': clip_denoised,
    'soft_limiter_on': str(soft_limiter_on),
    'soft_limiter_knee': soft_limiter_knee,
    'init_image': init_image,
    'init_scale': init_scale,
    'skip_steps': skip_steps,
    'animation_mode': animation_mode,
    'video_init_path':video_init_path,
    'extract_nth_frame_input':extract_nth_frame_input,
    'video_init_seed_continuity': str(video_init_seed_continuity),
    'key_frames': str(key_frames),
    'max_frames_input': max_frames_input,
    'interp_spline': interp_spline,
    'angle_input': angle_input,
    'zoom_input': zoom_input,
    'object_detection_zoom_input': object_detection_zoom_input,
    #'anim' try printing anim_steps and the series
    'translation_x_input': translation_x_input,
    'translation_y_input': translation_y_input,
    'translation_z_input': translation_z_input,
    'rotation_3d_x_input': rotation_3d_x_input,
    'rotation_3d_y_input': rotation_3d_y_input,
    'rotation_3d_z_input': rotation_3d_z_input,
    'midas_depth_model': midas_depth_model,
    'midas_weight_input': midas_weight_input,
    'near_plane_input': near_plane_input,
    'far_plane_input': far_plane_input,
    'fov_input': fov_input,
    'padding_mode': padding_mode,
    'sampling_mode': sampling_mode,
    'turbo_mode':str(turbo_mode),
    'turbo_steps':turbo_steps,
    'turbo_preroll':turbo_preroll,
    'frames_scale_input': frames_scale_input,
    'frames_skip_steps_input': frames_skip_steps_input,
    'vr_mode': str(vr_mode),
    'vr_eye_angle': vr_eye_angle,
    'vr_ipd': vr_ipd,
    'intermediate_saves': intermediate_saves,
    'intermediates_in_subfolder': str(intermediates_in_subfolder),
    'perlin_init': str(perlin_init),
    'perlin_mode': perlin_mode,
    'set_seed': set_seed,
    'seed': seed,
    'fuzzy_prompt': fuzzy_prompt,
    'rand_mag': rand_mag,
    'eta': eta,
    'clamp_grad': str(clamp_grad),
    'clamp_max_input': clamp_max_input,
    'cut_overview': str(cut_overview),
    'cut_innercut': str(cut_innercut),
    'cut_ic_pow': cut_ic_pow,
    'cut_icgray_p': str(cut_icgray_p),
    'use_vertical_symmetry':str(use_vertical_symmetry),
    'use_horizontal_symmetry':str(use_horizontal_symmetry),
    'transformation_percent':transformation_percent,
    'skip_video_for_run_all':skip_video_for_run_all,
    'make_regular_video':make_regular_video,
    'make_info_video':make_info_video,
    'make_unique_prompt_overlay_video':make_unique_prompt_overlay_video,
    'make_prompt_overlay_video':make_prompt_overlay_video,
    'make_interpolation_video':make_interpolation_video,
    'fps':fps,
    'mount_gdrive':mount_gdrive,
    'frames_to_interpolate':frames_to_interpolate,
    'gdrive_mountpoint':gdrive_mountpoint,
    'gdrive_subdirectory':gdrive_subdirectory,
    'number_of_interpolations':number_of_interpolations,
    'output_video':output_video,
    'output_video_fps':output_video_fps,
    'should_create_gist':should_create_gist,
    'github_username':github_username,
    'ccmixter_search_url_or_keyword':ccmixter_search_url_or_keyword,
    'finished_video_length':finished_video_length,    
    'show_first_prompt_in_youtube_description':show_first_prompt_in_youtube_description,
 
  }
  file_path=f"{batchFolder}/{batch_name}({batchNum})_settings.txt"
  if not save_to_project_folder:
    file_path=f"/content/drive/MyDrive/AI/settings_files/{batch_name}({batchNum})_settings.txt"
  #remove one item from the dictionary
  if not antarctic_prompt_frames_input:
    del setting_list['updated_antarctic_text_prompts']
  #if f"{batchFolder}/{batch_name}({batchNum})_settings.txt" exists, then delete it
  if os.path.exists(file_path):
    os.remove(file_path)
  # print('Settings:', setting_list)
  with open(file_path, "w+") as f:   #save settings
    json.dump(setting_list, f, ensure_ascii=False, indent=4)
  #how to add every item from a list to another

  
  


## 6. Define the secondary diffusion model

In [ ]:
#@title 1.6 Define the secondary diffusion model

def append_dims(x, n):
    return x[(Ellipsis, *(None,) * (n - x.ndim))]


def expand_to_planes(x, shape):
    return append_dims(x, len(shape)).repeat([1, 1, *shape[2:]])


def alpha_sigma_to_t(alpha, sigma):
    return torch.atan2(sigma, alpha) * 2 / math.pi


def t_to_alpha_sigma(t):
    return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)


@dataclass
class DiffusionOutput:
    v: torch.Tensor
    pred: torch.Tensor
    eps: torch.Tensor


class ConvBlock(nn.Sequential):
    def __init__(self, c_in, c_out):
        super().__init__(
            nn.Conv2d(c_in, c_out, 3, padding=1),
            nn.ReLU(inplace=True),
        )


class SkipBlock(nn.Module):
    def __init__(self, main, skip=None):
        super().__init__()
        self.main = nn.Sequential(*main)
        self.skip = skip if skip else nn.Identity()

    def forward(self, input):
        return torch.cat([self.main(input), self.skip(input)], dim=1)


class FourierFeatures(nn.Module):
    def __init__(self, in_features, out_features, std=1.):
        super().__init__()
        assert out_features % 2 == 0
        self.weight = nn.Parameter(torch.randn([out_features // 2, in_features]) * std)

    def forward(self, input):
        f = 2 * math.pi * input @ self.weight.T
        return torch.cat([f.cos(), f.sin()], dim=-1)


class SecondaryDiffusionImageNet(nn.Module):
    def __init__(self):
        super().__init__()
        c = 64  # The base channel count

        self.timestep_embed = FourierFeatures(1, 16)

        self.net = nn.Sequential(
            ConvBlock(3 + 16, c),
            ConvBlock(c, c),
            SkipBlock([
                nn.AvgPool2d(2),
                ConvBlock(c, c * 2),
                ConvBlock(c * 2, c * 2),
                SkipBlock([
                    nn.AvgPool2d(2),
                    ConvBlock(c * 2, c * 4),
                    ConvBlock(c * 4, c * 4),
                    SkipBlock([
                        nn.AvgPool2d(2),
                        ConvBlock(c * 4, c * 8),
                        ConvBlock(c * 8, c * 4),
                        nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                    ]),
                    ConvBlock(c * 8, c * 4),
                    ConvBlock(c * 4, c * 2),
                    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                ]),
                ConvBlock(c * 4, c * 2),
                ConvBlock(c * 2, c),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ]),
            ConvBlock(c * 2, c),
            nn.Conv2d(c, 3, 3, padding=1),
        )

    def forward(self, input, t):
        timestep_embed = expand_to_planes(self.timestep_embed(t[:, None]), input.shape)
        v = self.net(torch.cat([input, timestep_embed], dim=1))
        alphas, sigmas = map(partial(append_dims, n=v.ndim), t_to_alpha_sigma(t))
        pred = input * alphas - v * sigmas
        eps = input * sigmas + v * alphas
        return DiffusionOutput(v, pred, eps)


class SecondaryDiffusionImageNet2(nn.Module):
    def __init__(self):
        super().__init__()
        c = 64  # The base channel count
        cs = [c, c * 2, c * 2, c * 4, c * 4, c * 8]

        self.timestep_embed = FourierFeatures(1, 16)
        self.down = nn.AvgPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

        self.net = nn.Sequential(
            ConvBlock(3 + 16, cs[0]),
            ConvBlock(cs[0], cs[0]),
            SkipBlock([
                self.down,
                ConvBlock(cs[0], cs[1]),
                ConvBlock(cs[1], cs[1]),
                SkipBlock([
                    self.down,
                    ConvBlock(cs[1], cs[2]),
                    ConvBlock(cs[2], cs[2]),
                    SkipBlock([
                        self.down,
                        ConvBlock(cs[2], cs[3]),
                        ConvBlock(cs[3], cs[3]),
                        SkipBlock([
                            self.down,
                            ConvBlock(cs[3], cs[4]),
                            ConvBlock(cs[4], cs[4]),
                            SkipBlock([
                                self.down,
                                ConvBlock(cs[4], cs[5]),
                                ConvBlock(cs[5], cs[5]),
                                ConvBlock(cs[5], cs[5]),
                                ConvBlock(cs[5], cs[4]),
                                self.up,
                            ]),
                            ConvBlock(cs[4] * 2, cs[4]),
                            ConvBlock(cs[4], cs[3]),
                            self.up,
                        ]),
                        ConvBlock(cs[3] * 2, cs[3]),
                        ConvBlock(cs[3], cs[2]),
                        self.up,
                    ]),
                    ConvBlock(cs[2] * 2, cs[2]),
                    ConvBlock(cs[2], cs[1]),
                    self.up,
                ]),
                ConvBlock(cs[1] * 2, cs[1]),
                ConvBlock(cs[1], cs[0]),
                self.up,
            ]),
            ConvBlock(cs[0] * 2, cs[0]),
            nn.Conv2d(cs[0], 3, 3, padding=1),
        )

    def forward(self, input, t):
        timestep_embed = expand_to_planes(self.timestep_embed(t[:, None]), input.shape)
        v = self.net(torch.cat([input, timestep_embed], dim=1))
        alphas, sigmas = map(partial(append_dims, n=v.ndim), t_to_alpha_sigma(t))
        pred = input * alphas - v * sigmas
        eps = input * sigmas + v * alphas
        return DiffusionOutput(v, pred, eps)

# **Pre Video Settings Input**

## **Pre Diffuse Settings**

### **Settings Overwrite**

In [ ]:
import os
from IPython.display import display as dropdown_widget_display
import ipywidgets as widgets
settings_file_directory="/content/drive/MyDrive/AI/settings_files/"
folder_files = os.listdir(settings_file_directory) #You can also use full path.

folder_files=[]
folder_files.append('custom directory')
for item in os.listdir(settings_file_directory) :
  folder_files.append(item)
scW = widgets.Select(options=folder_files)
dropdown_widget_display(scW)

In [ ]:
overwrite_settings_file_custom_directory = "none" #@param{type: 'string'}
if scW.value=='custom_directory':
    overwrite_settings_file = overwrite_settings_file_custom_directory
else:
    overwrite_settings_file = settings_file_directory + scW.value
    print(overwrite_settings_file)


### **Batch Name**

In [ ]:
batch_name = 'testing_many' #@param{type: 'string'}

### **Frame Multiplier**

In [ ]:


#@markdown This number is multiplied by every user input frame number
absolute_frame_multiplier = 1 #@param {type:"integer"}

### **Antarctic AI Prompts**

In [ ]:
#@markdown comma seperate frames that should have automatically generated prompts based on the previous frame image
#@markdown Example: 10,50,100,150
#@markdown if left blank then it won't set up Antarctic at all
antarctic_prompt_frames_input = "" #@param {type: 'string'}
antarctic_prompt_frames = antarctic_prompt_frames_input
#@markdown Example: pre and post strings will be placed after the generated caption
antarctic_pre_string = "" #@param {type: 'string'}
antarctic_post_string = ",trending on Artstation"#@param {type: 'string'}
antarctic_number_of_captions =  2#@param {type: 'number'}

### **Diffusion and CLIP Model Settings**
#### **Model Settings**

In [ ]:

diffusion_model = "512x512_diffusion_uncond_finetune_008100" #@param ["256x256_diffusion_uncond", "512x512_diffusion_uncond_finetune_008100"]
use_secondary_model = True #@param {type: 'boolean'}
diffusion_sampling_mode = 'ddim' #@param ['plms','ddim']  
use_checkpoint = True #@param {type: 'boolean'}
ViTB32 = True #@param{type:"boolean"}
ViTB16 = False #@param{type:"boolean"}
ViTL14 = False #@param{type:"boolean"}
ViTL14_336px = False #@param{type:"boolean"}
RN101 = False #@param{type:"boolean"}
RN50 = False #@param{type:"boolean"}
RN50x4 = False #@param{type:"boolean"}
RN50x16 = False #@param{type:"boolean"}
RN50x64 = False #@param{type:"boolean"}
#@markdown If you're having issues with model downloads, check this to compare SHA's:
check_model_SHA = False #@param{type:"boolean"}

### **Image/Frame Settings**
#### **Basic Settings:**

In [ ]:


steps =  20#@param [25,50,100,150,250,500,1000]{type: 'raw', allow-input: true}
width_height = [500,500]#@param{type: 'raw'}
clip_guidance_scale_input = "30000"#@param{type: 'string'}
tv_scale_input = "150000"#@param{type: 'string'}
range_scale_input = "90"#@param{type: 'string'}
sat_scale_input = "190000"#@param{type: 'string'}
cutn_batches_input = "2"#@param{type: 'string'}
skip_augs= False#@param{type: 'boolean'}



#@markdown ---
#@markdown ### **Soft Limiter (Use 0.97 - 0.995 range):**
#@markdown *Experimental! ...may help mitigate color clipping.*
soft_limiter_on = True#@param{type: 'boolean'}
soft_limiter_knee = .98 #@param{type: 'number'}



#@markdown ---
#@markdown ### **Init Settings:**
init_image = "None" #@param{type: 'string'}
init_scale = 1000 #@param{type: 'integer'}
skip_steps =  15#@param{type: 'integer'}
#@markdown *Make sure you set skip_steps to ~50% of your steps if you want to use an init image.*


### **Animation Settings**
#### **Animation Mode:**

In [ ]:
animation_mode = '2D' #@param ['None', '2D', '3D', 'Video Input'] {type:'string'}
#@markdown *For animation, you probably want to turn `cutn_batches` to 1 to make it quicker.*













#======= VR MODE
#@markdown ---
#@markdown ####**VR Mode (3D anim only):**
#@markdown Enables stereo rendering of left/right eye views (supporting Turbo) which use a different (fish-eye) camera projection matrix.   
#@markdown Note the images you're prompting will work better if they have some inherent wide-angle aspect
#@markdown The generated images will need to be combined into left/right videos. These can then be stitched into the VR180 format.
#@markdown Google made the VR180 Creator tool but subsequently stopped supporting it. It's available for download in a few places including https://www.patrickgrunwald.de/vr180-creator-download
#@markdown The tool is not only good for stitching (videos and photos) but also for adding the correct metadata into existing videos, which is needed for services like YouTube to identify the format correctly.
#@markdown Watching YouTube VR videos isn't necessarily the easiest depending on your headset. For instance Oculus have a dedicated media studio and store which makes the files easier to access on a Quest https://creator.oculus.com/manage/mediastudio/
#@markdown 
#@markdown The command to get ffmpeg to concat your frames for each eye is in the form: `ffmpeg -framerate 15 -i frame_%4d_l.png l.mp4` (repeat for r)

vr_mode = False #@param {type:"boolean"}
#@markdown `vr_eye_angle` is the y-axis rotation of the eyes towards the center
vr_eye_angle = 0.5 #@param{type:"number"}
#@markdown interpupillary distance (between the eyes)
vr_ipd = 5.0 #@param{type:"number"}

#### Video Input Settings

In [ ]:
#@markdown ---

#@markdown ###**Video Input Settings:**
if is_colab:
    video_init_path = "/content/training.mp4" #@param {type: 'string'}
else:
    video_init_path = "training.mp4" #@param {type: 'string'}
extract_nth_frame_input = 2 #@param {type: 'number'}
video_init_seed_continuity = False #@param {type: 'boolean'}

#### 2D Animation Settings

In [ ]:
#@markdown ---
#@markdown ####**2D Animation Settings:**
#@markdown `zoom` is a multiplier of dimensions, 1 is no zoom.
#@markdown All rotations are provided in degrees.
key_frames = True #@param {type:"boolean"}
max_frames_input = 5#@param {type:"number"}

interp_spline = 'Linear' #Do not change, currently will not look good. param ['Linear','Quadratic','Cubic']{type:"string"}
angle_input = "0: (0)"#@param {type:"string"}
zoom_input = "0: (1)"#@param {type:"string"}
object_detection_zoom_input = []#@param{type: 'raw'}
translation_x_input = "0: (0)"#@param {type:"string"}
translation_y_input = "0: (0)"#@param {type:"string"}
translation_z_input = "0: (0)"#@param {type:"string"}
rotation_3d_x_input = "0: (0)"#@param {type:"string"}
rotation_3d_y_input = "0: (0)"#@param {type:"string"}
rotation_3d_z_input = "0: (0), 100: (90)"#@param {type:"string"}
midas_depth_model = "dpt_large"#@param {type:"string"}
midas_weight_input = "0.3"#@param{type: 'string'}
near_plane_input = "200"#@param{type: 'string'}
far_plane_input = "10000"#@param{type: 'string'}
fov_input = "120"#@param{type: 'string'}
padding_mode = 'border'#@param {type:"string"}
sampling_mode = 'bicubic'#@param {type:"string"}

#### Turbo Mode

In [ ]:
#@markdown ---
#@markdown ####**Turbo Mode (3D anim only):**
#@markdown (Starts after frame 10,) skips diffusion steps and just uses depth map to warp images for skipped frames.
#@markdown Speeds up rendering by 2x-4x, and may improve image coherence between frames. frame_blend_mode smooths abrupt texture changes across 2 frames.
#@markdown For different settings tuned for Turbo Mode, refer to the original Disco-Turbo Github: https://github.com/zippy731/disco-diffusion-turbo

turbo_mode = True #@param {type:"boolean"}
turbo_steps = "6" #@param ["2","3","4","5","6"] {type:"string"}
#@markdown ---

#@markdown ####**Coherency Settings:**
#@markdown `frame_scale` tries to guide the new frame to looking like the old one. A good default is 1500.
frames_scale_input =  "1500"#@param{type: 'string'}
#@markdown `frames_skip_steps` will blur the previous frame - higher values will flicker less but struggle to add enough new detail to zoom into.
frames_skip_steps_input = "1: (85), 50: (65), 100: (85)" #@param{type: 'string'}

#### VR MODE

In [ ]:
#======= VR MODE
#@markdown ---
#@markdown ####**VR Mode (3D anim only):**
#@markdown Enables stereo rendering of left/right eye views (supporting Turbo) which use a different (fish-eye) camera projection matrix.   
#@markdown Note the images you're prompting will work better if they have some inherent wide-angle aspect
#@markdown The generated images will need to be combined into left/right videos. These can then be stitched into the VR180 format.
#@markdown Google made the VR180 Creator tool but subsequently stopped supporting it. It's available for download in a few places including https://www.patrickgrunwald.de/vr180-creator-download
#@markdown The tool is not only good for stitching (videos and photos) but also for adding the correct metadata into existing videos, which is needed for services like YouTube to identify the format correctly.
#@markdown Watching YouTube VR videos isn't necessarily the easiest depending on your headset. For instance Oculus have a dedicated media studio and store which makes the files easier to access on a Quest https://creator.oculus.com/manage/mediastudio/
#@markdown 
#@markdown The command to get ffmpeg to concat your frames for each eye is in the form: `ffmpeg -framerate 15 -i frame_%4d_l.png l.mp4` (repeat for r)

vr_mode = False #@param {type:"boolean"}
#@markdown `vr_eye_angle` is the y-axis rotation of the eyes towards the center
vr_eye_angle = 0.5 #@param{type:"number"}
#@markdown interpupillary distance (between the eyes)
vr_ipd = 5.0 #@param{type:"number"}


### **Extra Settings**
#### **Saving:**

In [ ]:

intermediate_saves = 0#@param{type: 'raw'}
intermediates_in_subfolder = True #@param{type: 'boolean'}
#@markdown Intermediate steps will save a copy at your specified intervals. You can either format it as a single integer or a list of specific steps 

#@markdown A value of `2` will save a copy at 33% and 66%. 0 will save none.

#@markdown A value of `[5, 9, 34, 45]` will save at steps 5, 9, 34, and 45. (Make sure to include the brackets)


#@markdown ---


#@markdown ####**Advanced Settings:**
#@markdown *There are a few extra advanced settings available if you double click this cell.*

#@markdown *Perlin init will replace your init, so uncheck if using one.*

perlin_init = False  #@param{type: 'boolean'}
perlin_mode = 'mixed' #@param ['mixed', 'color', 'gray']
set_seed = 'random_seed' #@param{type: 'string'}
eta_input = "0.9"#@param{type: 'string'}
clamp_grad = True #@param{type: 'boolean'}
clamp_max_input = "0.06"#@param{type: 'string'}





#### Cutn Scheduling and Transformation Settings

In [ ]:
#@markdown ---

#@markdown ####**Cutn Scheduling:**
#@markdown Format: `[40]*400+[20]*600` = 40 cuts for the first 400 /1000 steps, then 20 for the last 600/1000

#@markdown cut_overview and cut_innercut are cumulative for total cutn on any given step. Overview cuts see the entire image and are good for early structure, innercuts are your standard cutn.

cut_overview = "[20]*200+[16]*200+[10]*200+[8]*200+[2]*200" #@param {type: 'string'}       
cut_innercut ="[2]*200+[4]*200+[6]*200+[8]*200+[4]*200"#@param {type: 'string'}  
cut_ic_pow_input = "10"#@param{type: 'string'} 
cut_icgray_p = "[0.2]*400+[0]*800"#@param {type: 'string'}


#@markdown ---

#@markdown ####**Transformation Settings:**
use_vertical_symmetry = False #@param {type:"boolean"}
use_horizontal_symmetry = False #@param {type:"boolean"}
transformation_percent = [0.09] #@param

## Prompts
`animation_mode: None` will only use the first set. `animation_mode: 2D / Video` will run through them per the set frames and hold on the last one.

In [ ]:
#this comment is used to indicate the beginning of prompt creation DONT CHANGE!!!!

In [ ]:
text_prompts = {
    0: [
            "girl detailed portrait of an adorable egyptian baby girl by Nick Silva, Shin JeongHo, Wandah Kurniawan, Symmetrical composition with people centered, trending on artstation, colorized:4",
            "text:-2",
            "glasses:-1",
            "color:1"
        ],
    1: [
            "DÜNYA DÖNÜYOR detailed portrait of an adorable egyptian baby girl by Nick Silva, Shin JeongHo, Wandah Kurniawan, Symmetrical composition with people centered, trending on artstation, colorized:4",
            "text:-2",
            "glasses:-1",
            "color:1"
        ],
    2: [
            "guy detailed portrait of an adorable egyptian baby girl by Nick Silva, Shin JeongHo, Wandah Kurniawan, Symmetrical composition with people centered, trending on artstation, colorized:4",
            "text:-2",
            "glasses:-1",
            "color:1"
        ]


}

image_prompts = {
    # 0:['ImagePromptsWorkButArentVeryGood.png:2',],
}

In [ ]:
#this comment is used to indicate the end of prompt creation DONT CHANGE!!!!

### Prompt Helpers

In [ ]:
#add every item of a list to each prompt
# for key in text_prompts:
#   text_prompts[key].extend(["text:-5","words:-5"])

## Diffuse Settings

In [ ]:
#@markdown #**Diffuse!**
#@markdown ###**Do the Run!**
#@markdown `n_batches` ignored with animation modes.

display_rate =  250 #@param{type: 'number'}
n_batches =  200 #@param{type: 'number'}
#@markdown ---


resume_run = True #@param{type: 'boolean'}
run_to_resume = '-1' #@param{type: 'string'}
resume_from_frame = 'latest' #@param{type: 'string'}
retain_overwritten_frames = False #@param{type: 'boolean'}




## Settings Overwrite

In [ ]:
import json

if 'settings.txt' in overwrite_settings_file:
  json_file=open(overwrite_settings_file)
  data = json.load(json_file)
  if 'text_prompts' in data:
    text_prompts=data['text_prompts']
    text_prompts = {int(k):v for k,v in text_prompts.items()}
  if 'absolute_frame_multiplier' in data: absolute_frame_multiplier=data['absolute_frame_multiplier']
  if 'antarctic_prompt_frames_input' in data: antarctic_prompt_frames_input=data['antarctic_prompt_frames_input']
  if 'antarctic_pre_string' in data: antarctic_pre_string=data['antarctic_pre_string']
  if 'antarctic_post_string' in data: antarctic_post_string=data['antarctic_post_string']
  if 'antarctic_number_of_captions' in data: antarctic_number_of_captions=data['antarctic_number_of_captions']
  if 'diffusion_model' in data: diffusion_model=data['diffusion_model']
  if 'use_secondary_model' in data:
    if data['use_secondary_model'] == 'True': use_secondary_model = True
    else: use_secondary_model = False    
  if 'diffusion_sampling_mode' in data: diffusion_sampling_mode=data['diffusion_sampling_mode']
  if 'use_checkpoint' in data:
    if data['use_checkpoint'] == 'True': use_checkpoint = True
    else: use_checkpoint = False
  if 'ViTB32' in data:
    if data['ViTB32'] == 'True': ViTB32 = True
    else: ViTB32 = False
  if 'ViTB16' in data:
    if data['ViTB16'] == 'True': ViTB16 = True
    else: ViTB16 = False
  if 'ViTL14' in data:
    if data['ViTL14'] == 'True': ViTL14 = True
    else: ViTL14 = False
  if 'ViTL14_336px' in data:
    if data['ViTL14_336px'] == 'True': ViTL14_336px = True
    else: ViTL14_336px = False
  if 'RN101' in data:
    if data['RN101'] == 'True': RN101 = True
    else: RN101 = False
  if 'RN50' in data:
    if data['RN50'] == 'True': RN50 = True
    else: RN50 = False
  if 'RN50x4' in data:
    if data['RN50x4'] == 'True': RN50x4 = True
    else: RN50x4 = False    
  if 'RN50x16' in data:
    if data['RN50x16'] == 'True': RN50x16 = True
    else: RN50x16 = False
  if 'RN50x64' in data:
    if data['RN50x64'] == 'True': RN50x64 = True
    else: RN50x64 = False
  if 'check_model_SHA' in data: check_model_SHA=data['check_model_SHA']
  if 'steps' in data: steps=data['steps']
  if 'diffusion_steps' in data: diffusion_steps=data['diffusion_steps']
  if 'width' in data and 'height' in data: width_height=[data['width'],data['height']]
  if 'clip_guidance_scale_input' in data: clip_guidance_scale_input=data['clip_guidance_scale_input']
  if 'tv_scale_input' in data: tv_scale_input=data['tv_scale_input']
  if 'range_scale_input' in data: range_scale_input=data['range_scale_input']
  if 'sat_scale_input' in data: sat_scale_input=data['sat_scale_input']
  if 'cutn_batches_input' in data: cutn_batches_input=data['cutn_batches_input']
  if 'skip_augs' in data:
    if data['skip_augs'] == 'True': skip_augs = True
    else: skip_augs = False
  if 'soft_limiter_on' in data:
    if data['soft_limiter_on'] == 'True': soft_limiter_on = True
    else: soft_limiter_on = False
  if 'soft_limiter_knee' in data: soft_limiter_knee=data['soft_limiter_knee']
  if 'init_image' in data: init_image=data['init_image']
  if 'init_scale' in data: init_scale=data['init_scale']
  if 'skip_steps' in data: skip_steps=data['skip_steps']
  if 'animation_mode' in data: animation_mode=data['animation_mode']
  if 'video_init_path' in data: video_init_path=data['video_init_path']
  if 'extract_nth_frame_input' in data: extract_nth_frame_input=data['extract_nth_frame_input']
  if 'video_init_seed_continuity' in data:
    if data['video_init_seed_continuity'] == 'True': video_init_seed_continuity = True
    else: video_init_seed_continuity = False
  if 'key_frames' in data:
    if data['key_frames'] == 'True': key_frames = True
    else: key_frames = False
  if 'max_frames_input' in data: max_frames_input=data['max_frames_input']
  if 'interp_spline' in data: interp_spline=data['interp_spline']
  if 'angle_input' in data: angle_input=data['angle_input']
  if 'zoom_input' in data: zoom_input=data['zoom_input']
  if 'object_detection_zoom_input' in data: object_detection_zoom_input=data['object_detection_zoom_input']
  if 'translation_x_input' in data: translation_x_input=data['translation_x_input']
  if 'translation_y_input' in data: translation_y_input=data['translation_y_input']
  if 'translation_z_input' in data: translation_z_input=data['translation_z_input']
  if 'rotation_3d_x_input' in data: rotation_3d_x_input=data['rotation_3d_x_input']
  if 'rotation_3d_y_input' in data: rotation_3d_y_input=data['rotation_3d_y_input']
  if 'rotation_3d_z_input' in data: rotation_3d_z_input=data['rotation_3d_z_input']
  if 'midas_depth_model' in data: midas_depth_model=data['midas_depth_model']
  if 'midas_weight_input' in data: midas_weight_input=data['midas_weight_input']
  if 'near_plane_input' in data: near_plane_input=data['near_plane_input']
  if 'far_plane_input' in data: far_plane_input=data['far_plane_input']
  if 'fov_input' in data: fov_input=data['fov_input']
  if 'padding_mode' in data: padding_mode=data['padding_mode']
  if 'sampling_mode' in data: sampling_mode=data['sampling_mode'] 
  if 'turbo_mode' in data:
    if data['turbo_mode'] == 'True': turbo_mode = True
    else: turbo_mode = False
  if 'turbo_steps' in data: turbo_steps=data['turbo_steps']
  if 'turbo_preroll' in data: turbo_preroll=data['turbo_preroll']
  if 'frames_scale_input' in data: frames_scale_input=data['frames_scale_input']
  if 'frames_skip_steps_input' in data: frames_skip_steps_input=data['frames_skip_steps_input']
  if 'vr_mode' in data:
    if data['vr_mode'] == 'True': vr_mode = True
    else: vr_mode = False
  if 'vr_eye_angle' in data: vr_eye_angle=data['vr_eye_angle']
  if 'vr_ipd' in data: vr_ipd=data['vr_ipd']
  if 'intermediate_saves' in data: intermediate_saves=data['intermediate_saves']
  if 'intermediates_in_subfolder' in data:
    if data['intermediates_in_subfolder'] == 'True': intermediates_in_subfolder = True
    else: intermediates_in_subfolder = False   
  if 'perlin_init' in data:
    if data['perlin_init'] == 'True': perlin_init = True
    else: perlin_init = False
  if 'perlin_mode' in data: perlin_mode=data['perlin_mode']
  if 'set_seed' in data: set_seed=data['set_seed']
  if 'fuzzy_prompt' in data: fuzzy_prompt=data['fuzzy_prompt']
  if 'rand_mag' in data: rand_mag=data['rand_mag']
  if 'eta' in data: eta=data['eta']
  if 'randomize_class' in data: randomize_class=data['randomize_class']
  if 'clip_denoised' in data: clip_denoised=data['clip_denoised']
  if 'clamp_grad' in data:
    if data['clamp_grad'] == 'True': clamp_grad = True
    else: clamp_grad = False    
  if 'clamp_max_input' in data: clamp_max_input=data['clamp_max_input']
  if 'cut_overview' in data: cut_overview=data['cut_overview']
  if 'cut_innercut' in data: cut_innercut=data['cut_innercut']
  if 'cut_ic_pow' in data: cut_ic_pow=data['cut_ic_pow']
  if 'cut_icgray_p' in data: cut_icgray_p=data['cut_icgray_p']
  if 'use_horizontal_symmetry' in data:
    if data['use_horizontal_symmetry'] == 'True': use_horizontal_symmetry = True
    else: use_horizontal_symmetry = False
  if 'use_vertical_symmetry' in data:
    if data['use_vertical_symmetry'] == 'True': use_vertical_symmetry = True
    else: use_vertical_symmetry = False
  if 'transformation_percent' in data: transformation_percent=data['transformation_percent']
  
  #absolute_frame_multiplier = ""
  #antarctic_prompt_frames_input = ""
  #antarctic_pre_string =""
  #antarctic_post_string =""
  #antarctic_number_of_captions =""
  #diffusion_model =""
  #use_secondary_model = True
  #diffusion_sampling_mode=""
  #use_checkpoint = True
  #ViTB32 = True
  #ViTB16 = True
  #ViTL14 = True
  #ViTL14_336px = True
  #RN101 = True
  #RN50 = True
  #RN50x4 = True
  #RN50x16 = True
  #RN50x64 = True
  #check_model_SHA=""
  #steps=""
  #diffusion_steps=""
  #width_height=[]
  #clip_guidance_scale_input=""
  #tv_scale_input=""
  #range_scale_input=""
  #sat_scale_input=""
  #cutn_batches_input=""
  #skip_augs = True
  #soft_limiter_on = True
  #soft_limiter_knee=""
  #init_image=""
  #init_scale=""
  #skip_steps=""
  #animation_mode=""
  #video_init_path=""
  #extract_nth_frame_input=""
  #video_init_seed_continuity = True
  #max_frames_input=""
  #interp_spline=""
  #angle_input=""
  #zoom_input=""
  #object_detection_zoom_input=""
  #translation_x_input=""
  #translation_y_input=""
  #translation_z_input=""
  #rotation_3d_x_input=""
  #rotation_3d_y_input=""
  #rotation_3d_z_input=""
  #midas_depth_model=""
  #midas_weight_input=""
  #near_plane_input=""
  #far_plane_input=""
  #fov_input=""
  #padding_mode=""
  #sampling_mode="" 
  #turbo_mode = True
  #turbo_steps=""
  #turbo_preroll=""
  #frames_scale_input=""
  #frames_skip_steps_input=""
  #vr_mode = True
  #vr_eye_angle=""
  #vr_ipd=""
  #intermediate_saves=""
  #intermediates_in_subfolder = True
  #perlin_init = True
  #perlin_mode=""
  #set_seed=""
  #fuzzy_prompt=""
  #rand_mag=""
  #eta=""
  #randomize_class=""
  #clip_denoised=""
  #clamp_grad = True
  #clamp_max_input=""
  #cut_overview=""
  #cut_innercut=""
  #cut_ic_pow=""
  #cut_icgray_p=""
  #use_horizontal_symmetry = True
  #use_vertical_symmetry = True
  #transformation_percent=""

# Generation Preparation

## 3. Antarctic Prompts

In [ ]:
##############################


antarctic_additional_prompts = ["text:-2"]
#antarctic_prompt_frames is antarctic_prompt_frames_input with every number multiplied by absolute_frame_multiplier
#if there is a , in antarctic_prompt_frames_input then it will split it up and multiply each number by absolute_frame_multiplier
#if there is no , then it will just multiply the number by absolute_frame_multiplier
if antarctic_prompt_frames_input:
  if 'x' in antarctic_prompt_frames_input:
      antarctic_prompt_frames_num = antarctic_prompt_frames_input.replace('x', '')
      antarctic_prompt_frames_mult = int(antarctic_prompt_frames_num) * absolute_frame_multiplier
      antarctic_prompt_frames = "x"+str(antarctic_prompt_frames_mult)
  elif "," in antarctic_prompt_frames_input:
      antarctic_prompt_frames = [int(int(i) * absolute_frame_multiplier) for i in antarctic_prompt_frames_input.split(",")]
      antarctic_prompt_frames = ",".join(str(i) for i in antarctic_prompt_frames)
  else:
      antarctic_prompt_frames = [int(int(antarctic_prompt_frames_input)) * absolute_frame_multiplier]
      antarctic_prompt_frames = ",".join(str(i) for i in antarctic_prompt_frames)

  

## 7. Diffusion and CLIP model settings

In [ ]:


model_256_SHA = '983e3de6f95c88c81b2ca7ebb2c217933be1973b1ff058776b970f901584613a'
model_512_SHA = '9c111ab89e214862b76e1fa6a1b3f1d329b1a88281885943d2cdbe357ad57648'
model_secondary_SHA = '983e3de6f95c88c81b2ca7ebb2c217933be1973b1ff058776b970f901584613a'

model_256_link = 'https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt'
model_512_link = 'https://v-diffusion.s3.us-west-2.amazonaws.com/512x512_diffusion_uncond_finetune_008100.pt'
model_secondary_link = 'https://v-diffusion.s3.us-west-2.amazonaws.com/secondary_model_imagenet_2.pth'

model_256_path = f'{model_path}/256x256_diffusion_uncond.pt'
model_512_path = f'{model_path}/512x512_diffusion_uncond_finetune_008100.pt'
model_secondary_path = f'{model_path}/secondary_model_imagenet_2.pth'

# Download the diffusion model
if diffusion_model == '256x256_diffusion_uncond':
  if os.path.exists(model_256_path) and check_model_SHA:
    print('Checking 256 Diffusion File')
    with open(model_256_path,"rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest();
    if hash == model_256_SHA:
      print('256 Model SHA matches')
      model_256_downloaded = True
    else: 
      print("256 Model SHA doesn't match, redownloading...")
      wget(model_256_link, model_path)
      model_256_downloaded = True
  elif os.path.exists(model_256_path) and not check_model_SHA or model_256_downloaded == True:
    print('256 Model already downloaded, check check_model_SHA if the file is corrupt')
  else:  
    wget(model_256_link, model_path)
    model_256_downloaded = True
elif diffusion_model == '512x512_diffusion_uncond_finetune_008100':
  if os.path.exists(model_512_path) and check_model_SHA:
    print('Checking 512 Diffusion File')
    with open(model_512_path,"rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest();
    if hash == model_512_SHA:
      print('512 Model SHA matches')
      model_512_downloaded = True
    else:  
      print("512 Model SHA doesn't match, redownloading...")
      wget(model_512_link, model_path)
      model_512_downloaded = True
  elif os.path.exists(model_512_path) and not check_model_SHA or model_512_downloaded == True:
    print('512 Model already downloaded, check check_model_SHA if the file is corrupt')
  else:  
    wget(model_512_link, model_path)
    model_512_downloaded = True


# Download the secondary diffusion model v2
if use_secondary_model == True:
  if os.path.exists(model_secondary_path) and check_model_SHA:
    print('Checking Secondary Diffusion File')
    with open(model_secondary_path,"rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest();
    if hash == model_secondary_SHA:
      print('Secondary Model SHA matches')
      model_secondary_downloaded = True
    else:  
      print("Secondary Model SHA doesn't match, redownloading...")
      wget(model_secondary_link, model_path)
      model_secondary_downloaded = True
  elif os.path.exists(model_secondary_path) and not check_model_SHA or model_secondary_downloaded == True:
    print('Secondary Model already downloaded, check check_model_SHA if the file is corrupt')
  else:  
    wget(model_secondary_link, model_path)
    model_secondary_downloaded = True

model_config = model_and_diffusion_defaults()
if diffusion_model == '512x512_diffusion_uncond_finetune_008100':
    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': 1000, #No need to edit this, it is taken care of later.
        'rescale_timesteps': True,
        'timestep_respacing': 250, #No need to edit this, it is taken care of later.
        'image_size': 512,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 256,
        'num_head_channels': 64,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_checkpoint': use_checkpoint,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })
elif diffusion_model == '256x256_diffusion_uncond':
    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': 1000, #No need to edit this, it is taken care of later.
        'rescale_timesteps': True,
        'timestep_respacing': 250, #No need to edit this, it is taken care of later.
        'image_size': 256,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 256,
        'num_head_channels': 64,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_checkpoint': use_checkpoint,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })

model_default = model_config['image_size']



if use_secondary_model:
    secondary_model = SecondaryDiffusionImageNet2()
    secondary_model.load_state_dict(torch.load(f'{model_path}/secondary_model_imagenet_2.pth', map_location='cpu'))
    secondary_model.eval().requires_grad_(False).to(device)

clip_models = []
if ViTB32: clip_models.append(clip.load('ViT-B/32', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTB16: clip_models.append(clip.load('ViT-B/16', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTL14: clip_models.append(clip.load('ViT-L/14', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTL14_336px: clip_models.append(clip.load('ViT-L/14@336px', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50: clip_models.append(clip.load('RN50', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x4: clip_models.append(clip.load('RN50x4', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x16: clip_models.append(clip.load('RN50x16', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x64: clip_models.append(clip.load('RN50x64', jit=False)[0].eval().requires_grad_(False).to(device))
if RN101: clip_models.append(clip.load('RN101', jit=False)[0].eval().requires_grad_(False).to(device))

normalize = T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
lpips_model = lpips.LPIPS(net='vgg').to(device)

## 8. Image/Frame Settings

In [ ]:
import re

def absolute_key_frame_multiplier(original_string):
    #if there is no colon in the string, return the original string
    if not re.search(':', original_string):
        return original_string
    #if original_string doesn't include decimals
    if not re.search('\.', original_string):
        multiplied_string = re.sub(r'\d+', lambda x: str(int(x.group(0)) * absolute_frame_multiplier), original_string)
        #print('mult',multiplied_string)
    #divide all numbers in parentheses by absolute_frame_multiplier
    # this brings them back to the original number since we only want to change frame numbers
        return re.sub(r'\((\d+)\)', lambda x: '(' + str(int(x.group(1)) // absolute_frame_multiplier) + ')', multiplied_string)
    else:
        #get all non-float numbers in the string
        non_floats = re.compile("(?<![\.\d])[0-9]+(?![\.\d])")
        #sub all non-floats with themselves times absolute_frame_multiplier
        return re.sub(non_floats, lambda x: str(int(x.group(0)) * absolute_frame_multiplier), original_string)

clip_guidance_scale = absolute_key_frame_multiplier(clip_guidance_scale_input)

tv_scale = absolute_key_frame_multiplier(tv_scale_input)

range_scale = absolute_key_frame_multiplier(range_scale_input)

sat_scale = absolute_key_frame_multiplier(sat_scale_input)

cutn_batches = absolute_key_frame_multiplier(cutn_batches_input)

if soft_limiter_knee < 0.5 or soft_limiter_knee > .999:
  soft_limiter_knee = .98
  print('soft_limiter_knee out of range. Automatically reset to 0.98')



#Get corrected sizes
side_x = (width_height[0]//64)*64;
side_y = (width_height[1]//64)*64;
if side_x != width_height[0] or side_y != width_height[1]:
  print(f'Changing output size to {side_x}x{side_y}. Dimensions must by multiples of 64.')

#Update Model Settings
timestep_respacing = f'ddim{steps}'
diffusion_steps = (1000//steps)*steps if steps < 1000 else steps
model_config.update({
    'timestep_respacing': timestep_respacing,
    'diffusion_steps': diffusion_steps,
})

#Make folder for batch
batchFolder = f'{outDirPath}/{batch_name}'
createPath(batchFolder)

### 8.1. Animation Settings

In [ ]:


extract_nth_frame = extract_nth_frame_input * absolute_frame_multiplier


if animation_mode == "Video Input":
  if is_colab:
      videoFramesFolder = f'/content/videoFrames'
  else:
      videoFramesFolder = f'videoFrames'
  createPath(videoFramesFolder)
  print(f"Exporting Video Frames (1 every {extract_nth_frame})...")
  try:
    for f in pathlib.Path(f'{videoFramesFolder}').glob('*.jpg'):
      f.unlink()
  except:
    print('')
  vf = f'select=not(mod(n\,{extract_nth_frame}))'
  subprocess.run(['ffmpeg', '-i', f'{video_init_path}', '-vf', f'{vf}', '-vsync', 'vfr', '-q:v', '2', '-loglevel', 'error', '-stats', f'{videoFramesFolder}/%04d.jpg'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  #!ffmpeg -i {video_init_path} -vf {vf} -vsync vfr -q:v 2 -loglevel error -stats {videoFramesFolder}/%04d.jpg


max_frames = max_frames_input * absolute_frame_multiplier

if animation_mode == "Video Input":
  max_frames = len(glob(f'{videoFramesFolder}/*.jpg'))


angle = absolute_key_frame_multiplier(angle_input)
zoom = absolute_key_frame_multiplier(zoom_input)
object_detection_zoom = object_detection_zoom_input
translation_x = absolute_key_frame_multiplier(translation_x_input)
translation_y = absolute_key_frame_multiplier(translation_y_input)
translation_z = absolute_key_frame_multiplier(translation_z_input)
rotation_3d_x = absolute_key_frame_multiplier(rotation_3d_x_input)
rotation_3d_y = absolute_key_frame_multiplier(rotation_3d_y_input)
rotation_3d_z = absolute_key_frame_multiplier(rotation_3d_z_input)
midas_weight = absolute_key_frame_multiplier(midas_weight_input)
near_plane = absolute_key_frame_multiplier(near_plane_input)
far_plane = absolute_key_frame_multiplier(far_plane_input)
fov = absolute_key_frame_multiplier(fov_input)

#======= TURBO MODE

turbo_preroll = 10 # frames

#insist turbo be used only w 3d anim.
if turbo_mode and animation_mode != '3D':
  print('=====')
  print('Turbo mode only available with 3D animations. Disabling Turbo.')
  print('=====')
  turbo_mode = False


frames_scale = absolute_key_frame_multiplier(frames_scale_input)

frames_skip_steps = absolute_key_frame_multiplier(frames_skip_steps_input)




#insist VR be used only w 3d anim.
if vr_mode and animation_mode != '3D':
  print('=====')
  print('VR mode only available with 3D animations. Disabling VR.')
  print('=====')
  vr_mode = False

def parse_key_frames(string, prompt_parser=None):
    """Given a string representing frame numbers paired with parameter values at that frame,
    return a dictionary with the frame numbers as keys and the parameter values as the values.

    Parameters
    ----------
    string: string
        Frame numbers paired with parameter values at that frame number, in the format
        'framenumber1: (parametervalues1), framenumber2: (parametervalues2), ...'
    prompt_parser: function or None, optional
        If provided, prompt_parser will be applied to each string of parameter values.
    
    Returns
    -------
    dict
        Frame numbers as keys, parameter values at that frame number as values

    Raises
    ------
    RuntimeError
        If the input string does not match the expected format.
    
    Examples
    --------
    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)")
    {10: 'Apple: 1| Orange: 0', 20: 'Apple: 0| Orange: 1| Peach: 1'}

    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)", prompt_parser=lambda x: x.lower()))
    {10: 'apple: 1| orange: 0', 20: 'apple: 0| orange: 1| peach: 1'}
    """
    #import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param

    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames
    
animations_with_keyframes_higher_than_max = []
def get_inbetweens(name, key_frames, between_max_frames, integer=False):
    """Given a dict with frame numbers as keys and a parameter value as values,
    return a pandas Series containing the value of the parameter at every frame from 0 to max_frames.
    Any values not provided in the input dict are calculated by linear interpolation between
    the values of the previous and next provided frames. If there is no previous provided frame, then
    the value is equal to the value of the next provided frame, or if there is no next provided frame,
    then the value is equal to the value of the previous provided frame. If no frames are provided,
    all frame values are NaN.

    Parameters
    ----------
    key_frames: dict
        A dict with integer frame numbers as keys and numerical values of a particular parameter as values.
    integer: Bool, optional
        If True, the values of the output series are converted to integers.
        Otherwise, the values are floats.
    
    Returns
    -------
    pd.Series
        A Series with length max_frames representing the parameter values for each frame.
    
    Examples
    --------
    >>> max_frames = 5
    >>> get_inbetweens({1: 5, 3: 6})
    0    5.0
    1    5.0
    2    5.5
    3    6.0
    4    6.0
    dtype: float64

    >>> get_inbetweens({1: 5, 3: 6}, integer=True)
    0    5
    1    5
    2    5
    3    6
    4    6
    dtype: int64
    """
    key_frame_series = pd.Series([np.nan for a in range(between_max_frames)])
    
    for i, value in key_frames.items():
        key_frame_series[i] = value
        if i>between_max_frames:
          animations_with_keyframes_higher_than_max.append(name)
    key_frame_series = key_frame_series.astype(float)
    
    interp_method = interp_spline

    if interp_method == 'Cubic' and len(key_frames.items()) <=3:
      interp_method = 'Quadratic'
    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'      
    
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[between_max_frames-1] = key_frame_series[key_frame_series.last_valid_index()]
    # key_frame_series = key_frame_series.interpolate(method=intrp_method,order=1, limit_direction='both')
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(),limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def split_prompts(prompts):
  prompt_series = pd.Series([np.nan for a in range(max_frames)])
  for i, prompt in prompts.items():
    prompt_series[i] = prompt
  # prompt_series = prompt_series.astype(str)
  prompt_series = prompt_series.ffill().bfill()
  return prompt_series

used_series = []

def try_to_get_inbetweens(name, string, between_max_frames):
    used_series.append(name) #needs work
    # if name == "new_eta" or name == "new_clamp_max":
    #   #import re
    #   #print('string1',string)
    #   string = re.sub(r"[-+]?(?:\d*\.\d+)", lambda x: str(int(float(x.group()) * 1000)), string)
    #   #print('string',string)
    #   try:
    #       in_betweens = get_inbetweens(name, parse_key_frames(string))
    #       #print('in_betweens', in_betweens)
    #       to_return = in_betweens.divide(other = 1000)
    #       #print('to_return', to_return)
    #       return to_return
    #   except RuntimeError as e:
    #       print(
    #           "WARNING: You have selected to use key frames, but you have not "
    #           "formatted `"+name+"` correctly for key frames.\n"
    #           "Attempting to interpret `"+name+"` as "
    #           f'"0: ({string})"\n'
    #           "Please read the instructions to find out how to use key frames "
    #           "correctly.\n"
    #       )
    #       string = f"0: ({string})"
    #       in_betweens = get_inbetweens(name, parse_key_frames(string))
    #       to_return = in_betweens.divide(other = 1000)
    #       return to_return    
    # else:
    try:
        return get_inbetweens(name, parse_key_frames(string), between_max_frames)
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `"+name+"` correctly for key frames.\n"
            "Attempting to interpret `"+name+"` as "
            f'"0: ({string})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        string = f"0: ({string})"
        return get_inbetweens(name, parse_key_frames(string), between_max_frames)

def con_try_to_get_inbetweens(name, string, between_max_frames):
    if ":" not in str(string): #tj test this
      string = "0: ("+str(string)+")"
      #print(string)
    else:
      used_series.append(name)
    if name == "new_eta" or name == "new_clamp_max":
      import re
      #print('string1',string)
      string = re.sub(r"[-+]?(?:\d*\.\d+)", lambda x: str(int(float(x.group()) * 1000)), string)
      #print('string',string)
      try:
          in_betweens = get_inbetweens(name, parse_key_frames(string), between_max_frames)
          #print('in_betweens', in_betweens)
          to_return = in_betweens.divide(other = 1000)
          #print('to_return', to_return)
          return to_return
      except RuntimeError as e:
          print(
              "WARNING: You have selected to use key frames, but you have not "
              "formatted `"+name+"` correctly for key frames.\n"
              "Attempting to interpret `"+name+"` as "
              f'"0: ({string})"\n'
              "Please read the instructions to find out how to use key frames "
              "correctly.\n"
          )
          string = f"0: ({string})"
          in_betweens = get_inbetweens(name, parse_key_frames(string), between_max_frames)
          to_return = in_betweens.divide(other = 1000)
          return to_return    
    else:
      try:
          return get_inbetweens(name, parse_key_frames(string), between_max_frames)
      except RuntimeError as e:
          print(
              "WARNING: You have selected to use key frames, but you have not "
              "formatted `"+name+"` correctly for key frames.\n"
              "Attempting to interpret `"+name+"` as "
              f'"0: ({string})"\n'
              "Please read the instructions to find out how to use key frames "
              "correctly.\n"
          )
          string = f"0: ({string})"
          return get_inbetweens(name, parse_key_frames(string), between_max_frames)

if key_frames:
    angle_series = try_to_get_inbetweens("angle", angle, max_frames)
    zoom_series = try_to_get_inbetweens("zoom", zoom, max_frames)
    translation_x_series = try_to_get_inbetweens("translation_x", translation_x, max_frames)
    translation_y_series = try_to_get_inbetweens("translation_y", translation_y, max_frames)
    translation_z_series = try_to_get_inbetweens("translation_z", translation_z, max_frames)
    rotation_3d_x_series = try_to_get_inbetweens("rotation_3d_x", rotation_3d_x, max_frames)
    rotation_3d_y_series = try_to_get_inbetweens("rotation_3d_y", rotation_3d_y, max_frames)
    rotation_3d_z_series = try_to_get_inbetweens("rotation_3d_z", rotation_3d_z, max_frames)
    new_cutn_batches_series = con_try_to_get_inbetweens("new_cutn_batches", cutn_batches, max_frames)
    new_clip_guidance_scale_series = con_try_to_get_inbetweens("new_clip_guidance_scale", clip_guidance_scale, max_frames)
    new_tv_scale_series = con_try_to_get_inbetweens("new_tv_scale", tv_scale, max_frames)
    new_range_scale_series = con_try_to_get_inbetweens("new_range_scale", range_scale, max_frames)
    new_sat_scale_series = con_try_to_get_inbetweens("new_sat_scale", sat_scale, max_frames)
    new_midas_weight_series = con_try_to_get_inbetweens("new_midas_weight", midas_weight, max_frames)
    new_near_plane_series = con_try_to_get_inbetweens("new_near_plane", near_plane, max_frames)
    new_far_plane_series = con_try_to_get_inbetweens("new_far_plane", far_plane, max_frames)
    new_fov_series = con_try_to_get_inbetweens("new_fov", fov, max_frames)
    new_frames_scale_series = con_try_to_get_inbetweens("new_frames_scale", frames_scale, max_frames)
    new_frames_skip_steps_series = con_try_to_get_inbetweens("new_frames_skip_steps", frames_skip_steps, max_frames)
#make a note of how to do thisss shit! https://stackoverflow.com/questions/23227927/how-do-i-move-master-back-several-commits-in-git
else:
    angle = float(angle)
    zoom = float(zoom)
    translation_x = float(translation_x)
    translation_y = float(translation_y)
    translation_z = float(translation_z)
    rotation_3d_x = float(rotation_3d_x)
    rotation_3d_y = float(rotation_3d_y)
    rotation_3d_z = float(rotation_3d_z)

### 8.2. Extra Settings
 Partial Saves, Advanced Settings, Cutn Scheduling

In [ ]:


if type(intermediate_saves) is not list:
  if intermediate_saves:
    steps_per_checkpoint = math.floor((steps - skip_steps - 1) // (intermediate_saves+1))
    steps_per_checkpoint = steps_per_checkpoint if steps_per_checkpoint > 0 else 1
    print(f'Will save every {steps_per_checkpoint} steps')
  else:
    steps_per_checkpoint = steps+10
else:
  steps_per_checkpoint = None

if intermediate_saves and intermediates_in_subfolder is True:
  partialFolder = f'{batchFolder}/partials'
  createPath(partialFolder)

eta = absolute_key_frame_multiplier(eta_input)

clamp_max = absolute_key_frame_multiplier(clamp_max_input)

### EXTRA ADVANCED SETTINGS:
randomize_class = True
clip_denoised = False
fuzzy_prompt = False
rand_mag = 0.05


cut_ic_pow = absolute_key_frame_multiplier(cut_ic_pow_input)


if key_frames:
  new_eta_series = con_try_to_get_inbetweens("new_eta", eta, max_frames)
  new_clamp_max_series = con_try_to_get_inbetweens("new_clamp_max",  clamp_max, max_frames)
  new_cut_ic_pow_series = con_try_to_get_inbetweens("new_cut_ic_pow",  cut_ic_pow, max_frames)

## Box Zoom Setup

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
!pip install jupyter_bbox_widget
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import json
import os
import base64
guided_zoom_widget = BBoxWidget(
image='/content/drive/MyDrive/AI/Disco_Diffusion/init_images/TimeToDisco(1)_0.png',
classes=['zoom'],
)#todo delete the stuff out of here that we dont actually need
box_zoom_variable_updates = {
  'translation_x_series': {},
  'translation_y_series': {},
  'zoom_series': {},
  'rot_mat_center_series': [],
  'zoom_series_list': [],
  'frames_scale_series_list': [],
  'frames_skip_steps_series_list': [],
}

def key_frame_multiplierx(original_string):#add next frame to these as an argument!!!!!!!!!!!!!!!
    #if there is no colon in the string, return the original string
    if not re.search(':', original_string):
        return original_string
    #if original_string doesn't include decimals
    if not re.search('\.', original_string):
        multiplied_string = re.sub(r'\d+', lambda x: str(int(x.group(0)) * absolute_frame_multiplier), original_string)
        #print('mult',multiplied_string)
    #divide all numbers in parentheses by absolute_frame_multiplier
    # this brings them back to the original number since we only want to change frame numbers
        return re.sub(r'\((\d+)\)', lambda x: '(' + str(int(x.group(1)) // absolute_frame_multiplier) + ')', multiplied_string)
    else:
        #get all non-float numbers in the string
        non_floats = re.compile("(?<![\.\d])[0-9]+(?![\.\d])")
        #sub all non-floats with themselves times absolute_frame_multiplier
        return re.sub(non_floats, lambda x: str(int(x.group(0)) * absolute_frame_multiplier), original_string)

def guided_widget_button_clicked(button):
  global box_zoom_variable_updates
  should_endless_zoom = False
  translate_box_to_center = True
  guided_zoom_widget.bboxes
  frames_to_center = 30
  if button == 'skip':
    should_endless_zoom = guided_zoom_widget.bboxes[0]['guided_skip_btn_endless_zoom_input']
    translate_box_to_center = guided_zoom_widget.bboxes[0]['guided_skip_btn_translate_box_to_center_input']
    frames_to_center = int(guided_zoom_widget.bboxes[0]['guided_skip_btn_frames_to_center_input'])
    guided_zoom_input = guided_zoom_widget.bboxes[0]['guided_skip_btn_zoom_input']
    guided_frames_scale_input = guided_zoom_widget.bboxes[0]['guided_skip_btn_frames_scale_input']
    guided_frames_skip_steps_input = guided_zoom_widget.bboxes[0]['guided_skip_btn_frames_skip_steps_input']
  elif button == 'submit':
    should_endless_zoom = guided_zoom_widget.bboxes[0]['guided_submit_btn_endless_zoom_input']
    translate_box_to_center = guided_zoom_widget.bboxes[0]['guided_submit_btn_translate_box_to_center_input']
    frames_to_center = int(guided_zoom_widget.bboxes[0]['guided_submit_btn_frames_to_center_input'])
    guided_zoom_input = guided_zoom_widget.bboxes[0]['guided_submit_btn_zoom_input']
    guided_frames_scale_input = guided_zoom_widget.bboxes[0]['guided_submit_btn_frames_scale_input']
    guided_frames_skip_steps_input = guided_zoom_widget.bboxes[0]['guided_submit_btn_frames_skip_steps_input']
  next_frame = len(glob(batchFolder+f"/{batch_name}({batchNum})_*.png"))
  # if should_endless_zoom:
  #   box_zoom_variable_updates['zoom_series_list'] = try_to_get_inbetweens('guided_zoom_input', guided_zoom_input, max_frames).tolist()
  # else:
  #   box_zoom_variable_updates['zoom_series_list'] = try_to_get_inbetweens('guided_zoom_input', guided_zoom_input, translate_box_to_center).tolist()
  #zoom_series.update(pd.Series(box_zoom_variable_updates['zoom_series_list'], index = list(range(next_frame, next_frame+frames_to_center))))

  #guided_frames_scale_input = "0:(1.0),100:(1.05)"
  # zoom_series = try_to_get_inbetweens("zoom", "0:(1.0),100:(1.05)", max_frames)
  # print('zoom_series', zoom_series.tolist())
  # print('guided_frames_scale_input', guided_frames_scale_input)

  # guided_frames_scale_list = try_to_get_inbetweens('guided_frames_scale_input', guided_frames_scale_input, max_frames).tolist()
  # print('guided_frames_scale_list', guided_frames_scale_list)
  
  print('guided_frames_scale_input', guided_frames_scale_input)
  guided_frames_scale_list = try_to_get_inbetweens('guided_frames_scale_input', guided_frames_scale_input, frames_to_center).tolist()
  print('guided_frames_scale_list', guided_frames_scale_list)
  new_frames_scale_series.update(pd.Series(guided_frames_scale_list, index = list(range(next_frame, next_frame+len(guided_frames_scale_list)))))
  guided_frames_skip_steps_series_list = try_to_get_inbetweens('guided_frames_skip_steps_input', guided_frames_skip_steps_input, frames_to_center).tolist()
  new_frames_skip_steps_series.update(pd.Series(guided_frames_skip_steps_series_list, index = list(range(next_frame, next_frame+len(guided_frames_skip_steps_series_list)))))
  # guided_zoom_widget.bboxes
  # print(guided_zoom_widget.bboxes)
  box_x = guided_zoom_widget.bboxes[0]['x']
  box_y = guided_zoom_widget.bboxes[0]['y']
  box_width = guided_zoom_widget.bboxes[0]['width']
  box_height = guided_zoom_widget.bboxes[0]['height']
  print('box dims', box_x, box_y, box_width, box_height)
  zoom_scale = determine_zoom_scale(box_width, box_height, box_x, box_y, frames_to_center)
  zoom_scale_list = [zoom_scale for _ in range(frames_to_center)]
  zoom_series.update(pd.Series(zoom_scale_list, index = list(range(next_frame, next_frame+len(zoom_scale_list)))))
  #zoom_scale only works if we are zooming in and not doing endless zoom because it decides what the zoom should be 
  # to make the box area be equal to the area of the whole image. 
  box_center_x = box_x + box_width/2
  box_center_y = box_y + box_height/2
  x_translate_per_iteration = int(((args.width_height[0]//2) - box_center_x) / frames_to_center)
  y_translate_per_iteration = int(((args.width_height[1]//2) - box_center_y) / frames_to_center)  
  guided_translation_x_list = [x_translate_per_iteration for _ in range(frames_to_center)]
  translation_x_series.update(pd.Series(guided_translation_x_list, index = list(range(next_frame, next_frame+len(guided_translation_x_list)))))
  guided_translation_y_list = [y_translate_per_iteration for _ in range(frames_to_center)]
  translation_y_series.update(pd.Series(guided_translation_y_list, index = list(range(next_frame, next_frame+len(guided_translation_y_list)))))  
  box_zoom_variable_updates['rot_mat_center_series'] = [() for _ in range(max_frames + 30)]
  #find of if this is the code to make a list of a bunch of ints that are all the same so we can add it to the pd series
  for frame in range(next_frame, next_frame+frames_to_center):
    iteration_number = frame + 1 - next_frame
    # box_zoom_variable_updates['translation_x_series'][frame] = x_translate_per_iteration
    # box_zoom_variable_updates['translation_y_series'][frame] = y_translate_per_iteration
    # box_zoom_variable_updates['zoom_series'][frame] = zoom_scale
    box_zoom_variable_updates['rot_mat_center_series'][frame] = (box_center_x+(iteration_number*x_translate_per_iteration), box_center_y+(iteration_number*y_translate_per_iteration))

def on_submit():
  guided_widget_button_clicked('submit')

def on_skip():
  guided_widget_button_clicked('skip')

guided_zoom_widget.on_submit(on_submit)
guided_zoom_widget.on_skip(on_skip)


# Box Zoom Widget

In [ ]:
empty_label = widgets.Label(value=" ")
skip_header_label = widgets.Label(value="SKIP")
submit_header_label = widgets.Label(value="SUBMIT")
guided_skip_btn_endless_zoom_checkbox = widgets.Checkbox(value=False,description='Endless zoom',disabled=False,indent=False)
guided_zoom_widget.attach(guided_skip_btn_endless_zoom_checkbox, name='guided_skip_btn_endless_zoom_input')
guided_submit_btn_endless_zoom_checkbox = widgets.Checkbox(value=False,description='Endless zoom',disabled=False,indent=False)
guided_zoom_widget.attach(guided_submit_btn_endless_zoom_checkbox, name='guided_submit_btn_endless_zoom_input')
guided_skip_btn_translate_box_to_center_checkbox = widgets.Checkbox(value=True,description='Translate to center',disabled=False,indent=False)
guided_zoom_widget.attach(guided_skip_btn_translate_box_to_center_checkbox, name='guided_skip_btn_translate_box_to_center_input')
guided_submit_btn_translate_box_to_center_checkbox = widgets.Checkbox(value=True,description='Translate to center',disabled=False,indent=False)
guided_zoom_widget.attach(guided_submit_btn_translate_box_to_center_checkbox, name='guided_submit_btn_translate_box_to_center_input')
guided_skip_btn_frames_to_center_textbox = widgets.Text(value='30',placeholder='Type something',description='Frames to center:',disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_skip_btn_frames_to_center_textbox, name='guided_skip_btn_frames_to_center_input')
guided_submit_btn_frames_to_center_textbox = widgets.Text(value='30',placeholder='Type something',description='Frames to center:',disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_submit_btn_frames_to_center_textbox, name='guided_submit_btn_frames_to_center_input')
guided_skip_btn_zoom_textbox = widgets.Text(value='0: (1.0), 30: (1.01)', placeholder='Type something', description='Guided zoom input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_skip_btn_zoom_textbox, name='guided_skip_btn_zoom_input')
guided_submit_btn_zoom_textbox = widgets.Text(value='0: (1.0), 30: (1.01)', placeholder='Type something', description='Guided zoom input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_submit_btn_zoom_textbox, name='guided_submit_btn_zoom_input')
guided_skip_btn_frames_scale_textbox = widgets.Text(value='0: (1000)', placeholder='Type something', description='Guided frames scale input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_skip_btn_frames_scale_textbox, name='guided_skip_btn_frames_scale_input')
guided_submit_btn_frames_scale_textbox = widgets.Text(value='0: (1000)', placeholder='Type something', description='Guided frames scale input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_submit_btn_frames_scale_textbox, name='guided_submit_btn_frames_scale_input')
guided_skip_btn_frames_skip_steps_textbox = widgets.Text(value='0: (60)', placeholder='Type something', description='Guided frames skip steps input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_skip_btn_frames_skip_steps_textbox, name='guided_skip_btn_frames_skip_steps_input')
guided_submit_btn_frames_skip_steps_textbox = widgets.Text(value='0: (60)', placeholder='Type something', description='Guided frames skip steps input:', disabled=False,style= {'description_width': 'initial'})
guided_zoom_widget.attach(guided_submit_btn_frames_skip_steps_textbox, name='guided_submit_btn_frames_skip_steps_input')
items = [guided_zoom_widget, empty_label, skip_header_label,submit_header_label,guided_skip_btn_endless_zoom_checkbox, guided_submit_btn_endless_zoom_checkbox,guided_skip_btn_translate_box_to_center_checkbox, guided_submit_btn_translate_box_to_center_checkbox,
        guided_skip_btn_frames_to_center_textbox,guided_submit_btn_frames_to_center_textbox, guided_skip_btn_zoom_textbox,guided_submit_btn_zoom_textbox,
        guided_skip_btn_frames_scale_textbox,guided_submit_btn_frames_scale_textbox ,guided_skip_btn_frames_skip_steps_textbox, guided_submit_btn_frames_skip_steps_textbox]
guided_zoom_widget_box = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 600px)"), disabled =False)
guided_zoom_widget_box

# Diffuse Preparation

In [ ]:


#Update Model Settings
timestep_respacing = f'ddim{steps}'
diffusion_steps = (1000//steps)*steps if steps < 1000 else steps
model_config.update({
    'timestep_respacing': timestep_respacing,
    'diffusion_steps': diffusion_steps,
})

batch_size = 1 

def move_files(start_num, end_num, old_folder, new_folder):
    for i in range(start_num, end_num):
        old_file = old_folder + f'/{batch_name}({batchNum})_{i:04}.png'
        new_file = new_folder + f'/{batch_name}({batchNum})_{i:04}.png'
        os.rename(old_file, new_file)


if retain_overwritten_frames is True:
  retainFolder = f'{batchFolder}/retained'
  createPath(retainFolder)


#skip_step_ratio = int(frames_skip_steps.rstrip("%")) / 100
#calc_frames_skip_steps = math.floor(steps * skip_step_ratio)

# calc_frames_skip_steps_series = []
# if len(anim_steps_series) > 0:
#   for anim_steps_item in anim_steps_series:
#     calc_frames_skip_steps_series.append(math.floor(anim_steps_item * skip_step_ratio))


calc_frames_skip_steps_series = []
if len(new_frames_skip_steps_series) > 0:
  for frames_skip_steps_item in new_frames_skip_steps_series:
    skip_step_ratio = int(frames_skip_steps_item) / 100
    calc_frames_skip_steps_series.append(math.floor(steps * skip_step_ratio))



# if steps <= calc_frames_skip_steps:
#   sys.exit("ERROR: You can't skip more steps than your total steps")

if resume_run:
  if run_to_resume == 'latest':
    try:
      batchNum
    except:
      batchNum = len(glob(f"{batchFolder}/{batch_name}(*)_settings.txt"))-1
  else:
    batchNum = int(run_to_resume)
  if resume_from_frame == 'latest':
    start_frame = len(glob(batchFolder+f"/{batch_name}({batchNum})_*.png"))
    if animation_mode != '3D' and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
      start_frame = start_frame - (start_frame % int(turbo_steps))
  else:
    start_frame = int(resume_from_frame)+1
    if animation_mode != '3D' and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
      start_frame = start_frame - (start_frame % int(turbo_steps))
    if retain_overwritten_frames is True:
      existing_frames = len(glob(batchFolder+f"/{batch_name}({batchNum})_*.png"))
      frames_to_save = existing_frames - start_frame
      print(f'Moving {frames_to_save} frames to the Retained folder')
      move_files(start_frame, existing_frames, batchFolder, retainFolder)
else:
  start_frame = 0
  batchNum = len(glob(batchFolder+"/*.txt"))
  while os.path.isfile(f"{batchFolder}/{batch_name}({batchNum})_settings.txt") is True or os.path.isfile(f"{batchFolder}/{batch_name}-{batchNum}_settings.txt") is True:
    batchNum += 1

print(f'Starting Run: {batch_name}({batchNum}) at frame {start_frame}')

if set_seed == 'random_seed':
    random.seed()
    seed = random.randint(0, 2**32)
    # print(f'Using seed: {seed}')
else:
    seed = int(set_seed)
updated_antarctic_text_prompts = text_prompts
#if f'{batchFolder}/antarctic_text_prompts.json' exists, use it
if os.path.isfile(f'{batchFolder}/antarctic_text_prompts.json'):
    with open(f'{batchFolder}/antarctic_text_prompts.json') as f:
        updated_antarctic_text_prompts = json.load(f)
args = {
    'batchNum': batchNum,
    'text_prompts': text_prompts,
    'prompts_series':split_prompts(text_prompts) if text_prompts else None,
    'image_prompts_series':split_prompts(image_prompts) if image_prompts else None,
    'seed': seed,
    'display_rate':display_rate,
    'n_batches':n_batches if animation_mode == 'None' else 1,
    'batch_size':batch_size,
    'batch_name': batch_name,
    'steps': steps,
    'diffusion_sampling_mode': diffusion_sampling_mode,
    'width_height': width_height,
    'clip_guidance_scale': clip_guidance_scale,
    'tv_scale': tv_scale,
    'range_scale': range_scale,
    'sat_scale': sat_scale,
    'cutn_batches': cutn_batches,
    'soft_limiter_on': soft_limiter_on,
    'soft_limiter_knee': soft_limiter_knee,
    'init_image': init_image,
    'init_scale': init_scale,
    'skip_steps': skip_steps,
    'side_x': side_x,
    'side_y': side_y,
    'timestep_respacing': timestep_respacing,
    'diffusion_steps': diffusion_steps,
    'animation_mode': animation_mode,
    'video_init_path': video_init_path,
    'extract_nth_frame': extract_nth_frame,
    'video_init_seed_continuity': video_init_seed_continuity,
    'key_frames': key_frames,
    'max_frames': max_frames if animation_mode != "None" else 1,
    'interp_spline': interp_spline,
    'start_frame': start_frame,
    'angle': angle,
    'zoom': zoom,
    'object_detection_zoom': object_detection_zoom,
    'translation_x': translation_x,
    'translation_y': translation_y,
    'translation_z': translation_z,
    'rotation_3d_x': rotation_3d_x,
    'rotation_3d_y': rotation_3d_y,
    'rotation_3d_z': rotation_3d_z,
    'midas_depth_model': midas_depth_model,
    'midas_weight': midas_weight,
    'near_plane': near_plane,
    'far_plane': far_plane,
    'fov': fov,
    'padding_mode': padding_mode,
    'sampling_mode': sampling_mode,
    'angle_series':angle_series,
    'zoom_series':zoom_series,
    'translation_x_series':translation_x_series,
    'translation_y_series':translation_y_series,
    'translation_z_series':translation_z_series,
    'rotation_3d_x_series':rotation_3d_x_series,
    'rotation_3d_y_series':rotation_3d_y_series,
    'rotation_3d_z_series':rotation_3d_z_series,
    'frames_scale': frames_scale,
    #'calc_frames_skip_steps': calc_frames_skip_steps,
    'skip_step_ratio': skip_step_ratio,
    # 'new_cutn_batches': new_cutn_batches,
    # 'new_clip_guidance_scale': new_clip_guidance_scale,
    # 'new_tv_scale': new_tv_scale,
    # 'new_range_scale': new_range_scale,
    # 'new_sat_scale': new_sat_scale,
    # 'new_midas_weight': new_midas_weight,
    # 'new_near_plane': new_near_plane,
    # 'new_far_plane': new_far_plane,
    # 'new_fov': new_fov,
    # 'new_frames_scale': new_frames_scale,
    # 'new_frames_skip_steps': new_frames_skip_steps,
    # 'new_eta': new_eta,
    # 'new_clamp_max': new_clamp_max,
    'cut_ic_pow': cut_ic_pow,
    'new_cutn_batches_series': new_cutn_batches_series,
    'new_clip_guidance_scale_series': new_clip_guidance_scale_series,
    'new_tv_scale_series': new_tv_scale_series,
    'new_range_scale_series': new_range_scale_series,
    'new_sat_scale_series': new_sat_scale_series,
    'new_midas_weight_series': new_midas_weight_series,
    'new_near_plane_series': new_near_plane_series,
    'new_far_plane_series': new_far_plane_series,
    'new_fov_series': new_fov_series,
    'new_frames_scale_series': new_frames_scale_series,
    'new_eta_series': new_eta_series,
    'new_clamp_max_series':new_clamp_max_series,
    'new_cut_ic_pow_series':new_cut_ic_pow_series,
    #'calc_frames_skip_steps': calc_frames_skip_steps,
    'calc_frames_skip_steps_series': calc_frames_skip_steps_series,
    'text_prompts': text_prompts,
    'image_prompts': image_prompts,
    'cut_overview': eval(cut_overview),
    'cut_innercut': eval(cut_innercut),
    'cut_ic_pow': cut_ic_pow,
    'cut_icgray_p': eval(cut_icgray_p),
    'intermediate_saves': intermediate_saves,
    'intermediates_in_subfolder': intermediates_in_subfolder,
    'steps_per_checkpoint': steps_per_checkpoint,
    'perlin_init': perlin_init,
    'perlin_mode': perlin_mode,
    'set_seed': set_seed,
    'eta': eta,
    'clamp_grad': clamp_grad,
    'clamp_max': clamp_max,
    'skip_augs': skip_augs,
    'randomize_class': randomize_class,
    'clip_denoised': clip_denoised,
    'fuzzy_prompt': fuzzy_prompt,
    'rand_mag': rand_mag,
    'use_vertical_symmetry': use_vertical_symmetry,
    'use_horizontal_symmetry': use_horizontal_symmetry,
    'transformation_percent': transformation_percent,
    'used_series': used_series,
    'animations_with_keyframes_higher_than_max': animations_with_keyframes_higher_than_max,
    'box_zoom_variable_updates': box_zoom_variable_updates,
}

args = SimpleNamespace(**args)

print('Prepping model...')
model, diffusion = create_model_and_diffusion(**model_config)
model.load_state_dict(torch.load(f'{model_path}/{diffusion_model}.pt', map_location='cpu'))
model.requires_grad_(False).eval().to(device)
for name, param in model.named_parameters():
    if 'qkv' in name or 'norm' in name or 'proj' in name:
        param.requires_grad_()
if model_config['use_fp16']:
    model.convert_to_fp16()

gc.collect()
torch.cuda.empty_cache()


# Video Settings

## **Create the video**
### **Create video**

In [ ]:
#@markdown Video file will save in the same folder as your images.

skip_video_for_run_all = False #@param {type: 'boolean'}
make_regular_video = False #@param {type: 'boolean'}
make_info_video = False #@param {type: 'boolean'}
make_custom_text_overlay_video = True #@param {type: 'boolean'}
use_unique_prompt_as_custom_overlay = True #@param {type: 'boolean'}
make_prompt_overlay_video = False #@param {type: 'boolean'}
make_interpolation_video = True #@param {type: 'boolean'}
folder = batch_name #@param
latest_run=batchNum
run = latest_run #@param
init_frame = 1#@param {type:"number"} This is the frame where the video will start
final_frame="final_frame"
last_frame = final_frame#@param {type:"number"} You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.
fps = 12#@param {type:"number"}


# view_video_in_cell = True #@param {type: 'boolean'}

### **Custom Overlay Text**

In [ ]:
famous_characters_text=[
    "Menes (3000 BC) Egyptian pharaoh who founded the First Dynasty",
    "Moses (1391-1271 BC) Key figure of Jewish history",
    "Zoroaster (1200 BC) Iranian prophet founded Zoroastrianism",
    "Homer (928 BC) Greek poet who wrote Iliad and Odyssey",
    "Cyrus the Great (602-530 BC) Founder of the Achaemenid Empire",
    "Mahavira (601 BC) Principal figure of Jainism.",
    "Lao Tzu (600 BC) Author of Tao Te Ching, founder of Taoism",
    "Buddha (563-483 BC) Founder of Buddhism.",
    "Confucius (551-479 BC) Chinese philosopher.",
    "Plato (424-348 BC) Greek philosopher.",
    "Aristotle (384-322 BC) Greek philosopher and polymath",
    "Alexander the Great (356-323 BC) King of Macedonia and military leader",
    "Euclid (325-265 BC) Greek mathematician",
    "King Ashoka (260-232 BC) Indian King",
    "Shih Huang Ti (259-210 BC) King of the state of Qin",
    "Julius Caesar (100 BC-44 BC) Roman ruler",
    "Augustus Caesar (63 BC-14) First Emperor of Rome",
    "Mencius (38-03BC) Chinese philosopher",
    "Jesus of Nazareth (5 BC-30 AD) Central figure of Christianity",
    "St. Paul (5-67) Christian missionary",
    "Ts'ai Lun (50-121) Inventor of paper",
    "Mani (216) Iranian founder of Manichaeism",
    "Constantine the Great (272-337) Roman Emperor",
    "St. Augustine (354-430) Influential Christian saint",
    "Justinian I  (482-565) Emperor of Eastern Roman Empire",
    "Emperor Wen of Sui (541-604) Founder of China's Sui Dynasty",
    "Muhammad (570-632) Prophet of Islam",
    "Umar ibn al-Khattab (584-644) Powerful Muslim Caliphate",
    "Charlemagne (742-814) First western Emperor since the fall of Rome",
    "William the Conqueror (1028-1087) First Norman King of England",
    "Pope Urban II (1042-1099) Influential Pope who",
    "Genghis Kahn (1162-1227) Leader of the Mongols",
    "Johann Gutenberg (1395-1468) Inventor of the printing press",
    "Christopher Columbus (1451-1506) Italian explorer landed in America",
    "Queen Isabella I (1451-1504) Queen of Castille",
    "Vasco da Gama (1460s-524) Portuguese explorer",
    "Nicoli Machiavelli (1469-1527) Italian diplomat, father of political science",
    "Francisco Pizarro (1471-1541) Spanish Conquistador",
    "Nicolaus Copernicus (1473-1543) Renaissance mathematician and astronomer",
    "Michelangelo (1475-1564) Renaissance sculptor, painter and architect",
    "Martin Luther (1483-1546) Sought to reform the Roman Catholic Church",
    "Hernando Cortes (1485-1547) Spanish Conquistador",
    "John Calvin (1509-27 May 1564) Christian theologian",
    "Queen Elizabeth I (1533-1603) Queen of England",
    "Francis Bacon (1561- 1626) Creator of the scientific method",
    "Galileo Galilei (1564-1642) Italian scientist",
    "William Shakespeare (1564-1616) English poet and playwright",
    "Johannes Kepler (1571-1630) German mathematician and astronomer, created laws of planetary motion",
    "William Harvey (1578-1657) English physician, blood circulation",
    "Rene Descartes (1596-1650) French philosopher and mathematician",
    "Oliver Cromwell (1599-1658) Leader of Parliamentarians in English civil war",
    "Antony van Leeuwenhoek (1632-1723) Dutch chemist, founder of microbiology",
    "John Locke (1632-1704) English political philosopher, theory of liberal democracy",
    "Isaac Newton (1642-1727) British mathematician and scientist",
    "Johann Sebastian Bach (1685-1750) Composer and organist",
    "Voltaire (1694-1778) Key figure of European Enlightenment, satirist",
    "Leonhard Euler (1707-1783) Swiss mathematician, calculus and graph theory",
    "Jean-Jacques Rousseau (1712-1778) French philosopher, author of Social Contract",
    "Peter the Great (1721-1725) Russian Emperor, expanded the Tsarist Empire",
    "Adam Smith (1723-1790) Scottish social philosopher, classical economics",
    "George Washington (1732-1799) 1st President of US",
    "James Watt (1736-1819) Scottish engineer, steam engine",
    "Antoine Laurent Lavoisier (1743-1794) French chemist and biologist, chemical revolution",
    "Thomas Jefferson (1743-1826) 3rd President of US, Declaration of Independence",
    "Edward Jenner (1749-1823) Developed the world's first vaccine",
    "John Dalton (1766-1844) English chemist and physicist, atomic theory",
    "Thomas Malthus (1766-1834) English scholar who raised concern over growing population",
    "Napoleon Bonaparte (1769-1821) French military and political leader",
    "Ludwig van Beethoven (1770-1827) German composer",
    "Louis Daguerre (1780-1851) French artist and photographer, inventor of the camera",
    "Simon Bolivar (1783-1830) Liberator of Latin American countries",
    "Michael Faraday (1791-1867) English scientist, electromagnetism and electrochemistry",
    "Charles Darwin (1809-1882) Scientist who proposed theory of evolution",
    "Karl Marx (1818-1883) German Communist philosopher",
    "William T.G. Morton (1819-1868) American dentist, pioneered anaesthetic",
    "Louis Pasteur (1822-1895) French biologist, cure for rabies and other infectious diseases",
    "Gregor Mendel (1822-1884) Czech/Austrian scientist, founded science of genetics",
    "Joseph Lister (1827-1912) British surgeon, sterilisation and antiseptic surgery",
    "James Clerk Maxwell (1831-1879) Scottish physicist, electromagnetism",
    "Nikolaus August Otto (1832-1891) German engineer, petrol engine",
    "Wilhelm Conrad Roentgen (1845-1923) German physicist, electromagnetic waves or X-rays",
    "Thomas Edison (1847-1931) Inventor and businessman, helped introduce electricity and light bulbs",
    "Alexander Graham Bell (1847-1922) Scottish inventor, telephone",
    "Sigmund Freud (1856-1939) Austrian neurologist, psychoanalysis and subconscious",
    "Max Planck (1858-1947) German theoretical physicist, Quantum physics",
    "Henry Ford (1863-1947) Owner of Ford, revolutionised mass-production",
    "Lenin (1870-1924) Leader of the Russian Revolution",
    "Orville and Wilbur Wright Orville (1871-1948) Created and flew the first aeroplane",
    "Ernest Rutherford (1871-1937) NZ born British physicist, atomic physics",
    "Guglielmo Marconi (1874-1937) Italian engineer, radio transmission",
    "Joseph Stalin (1878-1953) Absolute ruler of the Soviet Union",
    "Albert Einstein (1879-1955) German/ US scientist, Theory of Relativity",
    "Alexander Fleming (1881-1955) Scottish biologist who discovered penicillin",
    "Adolf Hitler (1889-1945) Dictator of Nazi Germany",
    "Mao Zedong (1893-1976) Leader of the Communist Revolution",
    "Werner Heisenberg (1900-1976) German theoretical physicist, Quantum mechanics",
    "Enrico Fermi (1901-1954) Italian-American physicist, nuclear reactor",
    "Gregory Pincus (1903-1967) American biologist, oral contraceptive pill",
    "John F. Kennedy (1917-1963) 38th President of the US, Cuban Missile Crisis",
    "Mikhail Gorbachev (1931-) Leader of Soviet Communist Party"
]


custom_overlay_text = {}

    
for i in range(400):
      if i % 4 == 0:
          custom_overlay_text[i] = famous_characters_text[i//4]  



## **FILM Interpolation**
### **1. Setup FILM Interpolation Workspace**

In [ ]:

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

mount_gdrive = True # @param{type:"boolean"}

frames_to_interpolate = "custom text" #@param ["standard", "custom text"]
if make_custom_text_overlay_video==False:
    frames_to_interpolate = "standard"
gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}



 
#@markdown 

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.



#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.

#@markdown If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames
#@markdown after interpolation will be: 
#@markdown $$2^k (n-1) -1$$

#recursive_interpolation_passes = 1 #@param{'type':'integer'}
number_of_interpolations =  4#@param{'type':'integer'}
#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}
output_video_fps = 30 #@param{'type':'number'}



## **Create Gist**

In [ ]:
should_create_gist=True #@param{type:'boolean'}
github_username="tjthejuggler" #@param{type:'string'}

## **Get Music**

In [ ]:
ccmixter_search_url_or_keyword="http://dig.ccmixter.org/search?searchp=spider" #@param{type:'string'}
automatically_choose_music=True #@param{type:'boolean'}

## **Change Speed of a Video**

In [ ]:
finished_video_length=60 #@param{type:"raw"}


## **Create Youtube Video**

In [ ]:
show_first_prompt_in_youtube_description=True #@param {type:"boolean"}

## Video Settings Overwrite

In [ ]:
if 'settings.txt' in overwrite_settings_file:
  json_file=open(overwrite_settings_file)
  data = json.load(json_file)
  if 'skip_video_for_run_all' in data: skip_video_for_run_all = data['skip_video_for_run_all']
  if 'make_regular_video' in data: make_regular_video = data['make_regular_video']
  if 'make_info_video' in data: make_info_video = data['make_info_video']
  if 'make_unique_prompt_overlay_video' in data: make_unique_prompt_overlay_video = data['make_unique_prompt_overlay_video'],
  if 'make_prompt_overlay_video' in data: make_prompt_overlay_video = data['make_prompt_overlay_video']
  if 'make_interpolation_video' in data: make_interpolation_video = data['make_interpolation_video']
  if 'fps' in data: fps = data['fps'] 
#skip_video_for_run_all = ""
#make_regular_video = ""
#make_info_video = ""
#make_unique_prompt_overlay_video = ""
#make_prompt_overlay_video = ""
#make_interpolation_video = ""
#fps = ""
save_settings() 

# Show Diffuse

In [ ]:
try:
  do_run()
except KeyboardInterrupt:
    pass
finally:
    print('Seed used:', seed)
    gc.collect()
    torch.cuda.empty_cache()

# Save Promt Generation Algorithm

In [ ]:
from requests import get
import os
import json
dir = os.path.dirname(os.path.realpath("_file_"))
dir = '/content/drive/MyDrive/Colab Notebooks'
filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
json_file=open(dir +'/' + filename)
data = json.load(json_file)
full_source_code_list=[]
for i in range(len(data["cells"])):
  for line in data["cells"][i]["source"]:
    full_source_code_list.append(line)
prompt_generator_list=[]
should_append_string_to_prompt_generator_list=False
for string in full_source_code_list:
  if '#this comment is used to indicate the end of prompt creation DONT CHANGE!!!!' in string:
    break
  if should_append_string_to_prompt_generator_list:
    prompt_generator_list.append(string)
  elif '#this comment is used to indicate the beginning of prompt creation DONT CHANGE!!!!' in string:
      should_append_string_to_prompt_generator_list=True
prompt_generator_file_path=f"{batchFolder}/{batch_name}({batchNum})_prompt_generator.txt"
with open(prompt_generator_file_path, "w") as output:
    output.writelines(prompt_generator_list)
    output.close()
if not os.path.exists("/content/drive/MyDrive/AI/Pop_Diffusion/all_prompt_generator.txt"):
 f=open("/content/drive/MyDrive/AI/Pop_Diffusion/all_prompt_generator.txt","w")

#check if the prompt generator is empty
if len(prompt_generator_list)>0:
  new_data_to_write = "---------------------------------------------\n"
  new_data_to_write += batch_name+"\n"
  new_data_to_write += "---------------------------------------------\n"
  for line in prompt_generator_list:
    new_data_to_write += line

  with open("/content/drive/MyDrive/AI/Pop_Diffusion/all_prompt_generator.txt", "r") as output:
      f=output.read()
  if not new_data_to_write in f:
    all_data_to_write = f + "\n"+new_data_to_write
    with open("/content/drive/MyDrive/AI/Pop_Diffusion/all_prompt_generator.txt", "w") as output:
      output.write(all_data_to_write)

# Video and post-production

## 11. Create the video

In [ ]:
from PIL import Image,ImageFont,ImageDraw

print('text_prompts_begin', text_prompts)

def get_unique_text_prompts(orig_dict):
    original_dict = orig_dict.copy()
    def find_largest_common_substring(arr):
        list_length = len(arr)
        reference_item = arr[0]
        reference_item_length = len(reference_item) 
        res = ""
        for i in range(reference_item_length):
            for j in range(i + 1, reference_item_length + 1):
                stem = reference_item[i:j]
                k = 1
                for k in range(1, list_length):
                    if stem not in arr[k]:
                        break
                if (k + 1 == list_length and len(res) < len(stem)):
                    res = stem 
        return res
    unique_text_prompts = {}
    for prompt in original_dict:
        original_dict[prompt] = ' '.join(original_dict[prompt])
    for i in range(2):
        text_prompts_list = list(original_dict.values())
        largest_common = find_largest_common_substring(text_prompts_list)
        for frame in original_dict:
            unique_text_prompts[frame] = original_dict[frame].replace(largest_common, '')
        original_dict = unique_text_prompts.copy()
    for item in unique_text_prompts:
        unique_text_prompts[item] = [unique_text_prompts[item]]
    return unique_text_prompts

using_labels = False
if skip_video_for_run_all == True:
  print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
  # import subprocess in case this cell is run without the above cells
  import subprocess
  from base64 import b64encode
  latest_run = batchNum
  
  label_folder = 'label'
  prompt_folder = 'prompt'
  unique_prompt_folder = 'unique_prompt'
  custom_overlay_folder='custom_overlay'
 
  final_frame = 'final_frame'
  frames = []


  if last_frame == 'final_frame':
    last_frame = len(glob(batchFolder+f"/{folder}({run})_*.png"))
    print(f'Total frames: {last_frame}')

  dict_of_series = {
    'cutn_batches': args.new_cutn_batches_series, 
    'clip_guidance': args.new_clip_guidance_scale_series,
    'tv_scale': args.new_tv_scale_series,
    'range_scale': args.new_range_scale_series,
    'sat_scale': args.new_sat_scale_series,  
    'midas_weight': args.new_midas_weight_series, 
    'frames_scale': args.new_frames_scale_series, 
    'eta': args.new_eta_series,
    'clamp_max': args.new_clamp_max_series,
    'cut_ic_pow': args.new_cut_ic_pow_series,
    'frames_skip_steps': calc_frames_skip_steps_series,
    'text_prompts': text_prompts,
  }
  print('text_prompts_begin2', text_prompts)
  custom_overlay_text = custom_overlay_text.copy()
  if make_custom_text_overlay_video and make_unique_prompt_overlay_video:
    custom_overlay_text = get_unique_text_prompts(text_prompts).copy()
    print('custom_overlay_text', custom_overlay_text)
    
  #print('text_prompts', text_prompts)
  #print('updated_antarctic_text_prompts',updated_antarctic_text_prompts)
  prompt_object = text_prompts.copy()
  if os.path.isfile(f'{batchFolder}/antarctic_text_prompts.json'):
    print('setting this')
    prompt_object = {int(key):value for key, value in updated_antarctic_text_prompts.items()}
    dict_of_series['text_prompts'] = prompt_object
  print('text_prompts', text_prompts)
  print('prompt_object', prompt_object)
  
  def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))
    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized
  
  def add_padding_to_image(image_name, image_index):
      image = cv2.imread(f"{outDirPath}/{folder}/{image_name}")
      old_image_height, old_image_width, channels = image.shape
      #if the image is not a perfect square then add padding to it
      if old_image_width > 500 :
        image = image_resize(image, width = 500)
        old_image_height, old_image_width, channels = image.shape
      if old_image_height != old_image_width:
        new_image_height = old_image_width
        new_image_width = old_image_width
        new_image = np.zeros((new_image_height, new_image_width, channels), np.uint8)
        new_image[0:old_image_height, 0:old_image_width] = image
        image = new_image
      old_image_height, old_image_width, channels = image.shape
      # create new image of desired size and color (blue) for padding        
      new_image_height = old_image_height + (old_image_height//24)
      new_image_width = 16*(new_image_height//9)
      print(old_image_height, old_image_width, new_image_height, new_image_width)
      color = (255,0,0)
      result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)
      # compute center offset
      x_center = (new_image_width - old_image_width) // 2 + (new_image_width-new_image_height) // 2
      y_center = (new_image_height - old_image_height) // 2
      # copy image image into center of result image
      result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = image      
      for dict_index, (key, value) in enumerate(dict_of_series.items()):
          if key == 'text_prompts':
            #get the highest key in text_prompts that is lower than image_index
            #key_index = next(key for key, value in dict_of_series["text_prompts"].items() if key <= image_index)
            this_value = dict_of_series["text_prompts"].get(max(k for k in dict_of_series["text_prompts"] if k <= image_index))
            this_prompt = this_value
            color_coverted = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            pil_image=Image.fromarray(color_coverted)
            draw = ImageDraw.Draw(pil_image)
            # font = ImageFont.truetype(<font-file>, <font-size>)
            font = ImageFont.truetype("arial.ttf", 36)
            # draw.text((x, y),"Sample Text",(r,g,b))
            draw.text((0, 0),str(this_prompt),(255,255,255),font=font)
         
            # cv2.putText(result, text=str(this_prompt), org=(10,30),
            #     fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.4, color=(255,255,255),
            #     thickness=1, lineType=cv2.LINE_AA)
          else:
            raw_value = value[image_index]
            #limit raw_value to 4 decimal places
            raw_value = round(raw_value, 4) 
            color_coverted = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            pil_image=Image.fromarray(color_coverted)
            draw = ImageDraw.Draw(pil_image)
            # font = ImageFont.truetype(<font-file>, <font-size>)
            font = ImageFont.truetype("arial.ttf", 36)
            # draw.text((x, y),"Sample Text",(r,g,b))
            draw.text((0, 0),key+ ": " + str(raw_value),(255,255,255),font=font)            
            # cv2.putText(result, text=key+ ": " + str(raw_value), org=(10,60+(dict_index*30)),
            #     fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(255,255,255),
            #     thickness=2, lineType=cv2.LINE_AA)
      # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      # cv2.imwrite(f"{outDirPath}/{folder}/{label_folder}/{image_name}", result)
      pil_image.show()
      pil_image.save(f"{outDirPath}/{folder}/{label_folder}/{image_name}")
  def add_prompt_to_image(image_name, image_index, unique_prompt):
    image = cv2.imread(f"{outDirPath}/{folder}/{image_name}")
    print('image_index', image_index)
    if unique_prompt:
      this_prompt = custom_overlay_text.get(max(k for k in dict_of_series["text_prompts"] if k <= image_index))
    else:
      this_prompt = dict_of_series["text_prompts"].get(max(k for k in dict_of_series["text_prompts"] if k <= image_index))
    print('this_prompt', this_prompt)
    for prompt_item_index, prompt in enumerate(this_prompt) :
      color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      pil_image=Image.fromarray(color_coverted)
      draw = ImageDraw.Draw(pil_image)
      # font = ImageFont.truetype(<font-file>, <font-size>)
      font = ImageFont.truetype("/content/drive/MyDrive/AI/font/arial.ttf", 36)
      # draw.text((x, y),"Sample Text",(r,g,b))
      print("type", type(this_prompt))
      draw.text((0, 0),' '.join(this_prompt),(255,255,255),font=font)            
      # cv2.putText(result, text=key+ ": " + str(raw_value), org=(10,60+(dict_index*30)),
      #     fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(255,255,255),
      #     thickness=2, lineType=cv2.LINE_AA)
      # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      # cv2.imwrite(f"{outDirPath}/{folder}/{label_folder}/{image_name}", result)
      
      
    if unique_prompt:
      pil_image.save(f"{outDirPath}/{folder}/{unique_prompt_folder}/{image_name}")
      
    else:
      pil_image.save(f"{outDirPath}/{folder}/{prompt_folder}/{image_name}")

    
  for video_index in range(4):
    if video_index == 0:
      if not make_regular_video:
        continue
      image_path = f"{outDirPath}/{folder}/{folder}({run})_%04d.png"
      filepath = f"{outDirPath}/{folder}/{folder}({run}).mp4"
    if video_index == 1:
      if not make_info_video:
        continue
      fps = 1
      isExist = os.path.exists(f"{outDirPath}/{folder}/{label_folder}")
      if not isExist:
        os.makedirs(f"{outDirPath}/{folder}/{label_folder}")
      image_count = 0
      for index,imagename in enumerate(os.listdir(f"{outDirPath}/{folder}")):
          if imagename.endswith(".png"):
            add_padding_to_image(imagename,image_count)
            image_count += 1
      image_path = f"{outDirPath}/{folder}/{label_folder}/{folder}({run})_%04d.png"
      filepath = f"{outDirPath}/{folder}/{folder}({run})_info.mp4"
    if video_index == 2: 
      if not make_prompt_overlay_video:
        continue
      isExist = os.path.exists(f"{outDirPath}/{folder}/{prompt_folder}")
      if not isExist:
        os.makedirs(f"{outDirPath}/{folder}/{prompt_folder}")
      image_count = 0
      for index,imagename in enumerate(os.listdir(f"{outDirPath}/{folder}")):
          if imagename.endswith(".png"):
            add_prompt_to_image(imagename,image_count,unique_prompt = False)
            image_count += 1
      image_path = f"{outDirPath}/{folder}/{prompt_folder}/{folder}({run})_%04d.png"
      filepath = f"{outDirPath}/{folder}/{folder}({run})_prompts.mp4"  
    if video_index == 3: 
      if not make_unique_prompt_overlay_video:
        continue
      isExist = os.path.exists(f"{outDirPath}/{folder}/{unique_prompt_folder}")
      if not isExist:
        os.makedirs(f"{outDirPath}/{folder}/{unique_prompt_folder}")
      image_count_unique = 0
      for index,imagename in enumerate(os.listdir(f"{outDirPath}/{folder}")):
          print('loopin', index)
          if imagename.endswith(".png"):
            add_prompt_to_image(imagename,image_count_unique,unique_prompt = True)
            image_count_unique += 1
      image_path = f"{outDirPath}/{folder}/{unique_prompt_folder}/{folder}({run})_%04d.png"
      filepath = f"{outDirPath}/{folder}/{folder}({run})_unique_prompts.mp4"
    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec',
        'png',
        '-r',
        str(fps),
        '-start_number',
        str(init_frame),
        '-i',
        image_path,
        '-frames:v',
        str(last_frame+1),
        '-c:v',
        'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt',
        'yuv420p',
        '-crf',
        '17',
        '-preset',
        'veryslow',
        filepath
    ]
    print('video_index1', video_index)
    process = subprocess.Popen(cmd, cwd=f'{batchFolder}', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    try:
      if process.returncode != 0:
          print(stderr)
          raise RuntimeError(stderr)
          #delete all images from label folder
          if video_index == 1:
            for file in os.listdir(f"{outDirPath}/{folder}/{label_folder}"):
              if file.endswith(".png"):
                os.remove(f"{outDirPath}/{folder}/{label_folder}/{file}")
      else:
          print('video_index2', video_index)
          print("The video is ready and saved to the images folder")
          #delete all images from label folder
          if video_index == 1:
            for file in os.listdir(f"{outDirPath}/{folder}/{label_folder}"):
              if file.endswith(".png"):
                os.remove(f"{outDirPath}/{folder}/{label_folder}/{file}")
    except:
      pass

    # if view_video_in_cell:
    #     mp4 = open(filepath,'rb').read()
    #     data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    #     display.HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>')
  
print('text_prompts_end', text_prompts)


## 11. FILM Interpolation

In [ ]:
# This top bit only neneds to be run once
# ...may as well set up the notebook to 
# assume the user is only going to run all
# of setup once.


##############################



###################
def predict_number_of_interpolation_frames(frame_number,interpolation_number):
    for i in range(interpolation_number):
        prediction_number=(frame_number)*2-1
        frame_number=prediction_number
    return prediction_number
print(predict_number_of_interpolation_frames(2,17))

# Optionally Mount GDrive
frames_dir = f'{outDirPath}/{folder}'
if frames_to_interpolate == "unique prompts":
  frames_dir = f'{outDirPath}/{folder}/{unique_prompt_folder}'

number_of_interpolated_frames = 0

if os.path.exists(frames_dir+'/interpolated_frames'):
  number_of_interpolated_frames=len(os.listdir(frames_dir+'/interpolated_frames'))
  print('real_frame_number', number_of_interpolated_frames)
#/content/drive/MyDrive/AI/Disco_Diffusion/images_out/testing_many/unique_prompt/interpolated_frames
numbers_of_frames_to_be_interpolated=0
for imagename in os.listdir(frames_dir):
    print(imagename)
    if imagename.endswith(".png"):
      print("has png")
      numbers_of_frames_to_be_interpolated += 1
should_do_interpolation=False
if predict_number_of_interpolation_frames(numbers_of_frames_to_be_interpolated,number_of_interpolations) > number_of_interpolated_frames:
    should_do_interpolation=True
print("predict_number_of_interpolation_frames(numbers_of_frames_to_be_interpolated,number_of_interpolations",predict_number_of_interpolation_frames(numbers_of_frames_to_be_interpolated,number_of_interpolations))
print("number_of_interpolated_frames",number_of_interpolated_frames)
if make_interpolation_video and should_do_interpolation:
  interpolation_start_time = time.time()
  from pathlib import Path
  import os

  drive_mounted = False
  gdrive_fpath = '.'
  local_path = '/content/'


  if mount_gdrive and not drive_mounted:
      from google.colab import drive
     

     
      gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
      try:
          if not mounted_at_beginning:
            drive.mount(gdrive_mountpoint, force_remount = True)
          !mkdir -p {gdrive_fpath}
          %cd {gdrive_fpath}
          local_path = gdrive_fpath
          drive_mounted = True
      except OSError:
          print(
              "\n\n-----[PYTTI-TOOLS]-------\n\n"
              "If you received a scary OSError and your drive"
              " was already mounted, ignore it."
              "\n\n-----[PYTTI-TOOLS]-------\n\n"
              )
          raise

  #####################

  # Perform rest of setup

  if not Path('./frame-interpolation').exists():
      !git clone https://github.com/pytti-tools/frame-interpolation

  try:
      import frame_interpolation
  except ModuleNotFoundError:
      %pip install -r ./frame-interpolation/requirements_colab.txt
      %pip install ./frame-interpolation

  #url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
  share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

  if not (Path(local_path) / 'saved_model').exists():
      !pip install --upgrade gdown
      !gdown --folder {share_id}

  # create default frame
  !mkdir -p frames

### Record Interpolation Observations

In [ ]:
#chooose a random number up to a billion
interpolation_random_number = random.randint(0,10000000000)


this_runs_interpolation_observation_dict={  
    "gpu":gpu_name,
    "width":width_height[0],
    "height": width_height[1],
    "frames": numbers_of_frames_to_be_interpolated,
    "interp_num": number_of_interpolations,
    "status": "failed",
}
with open("/content/drive/MyDrive/AI/Pop_Diffusion/interpolation_observations.txt") as f:
    full_interpolation_observation_dict = json.load(f)
full_interpolation_observation_dict[batch_name+"*"+str(interpolation_random_number)]=this_runs_interpolation_observation_dict
with open("/content/drive/MyDrive/AI/Pop_Diffusion/interpolation_observations.txt", "w+") as ant: 
    json.dump(full_interpolation_observation_dict, ant, ensure_ascii=False, indent=4)


 ### 11.2. Interpolate!

In [ ]:
if make_interpolation_video and should_do_interpolation:
  #@markdown
  #@markdown Results will be saved in a subdirectory of `frames_dir` named "interpolated_frames"

  from loguru import logger

  logger.info("Beginning interpolation...")
  
  if output_video:
    !python -m frame_interpolation.eval.interpolator_cli \
        --model_path ./saved_model \
        --pattern {frames_dir} \
        --fps {output_video_fps} \
        --times_to_interpolate {number_of_interpolations} \
        --output_video
        

  else:
      !python -m frame_interpolation.eval.interpolator_cli \
        --model_path ./saved_model \
        --pattern {frames_dir} \

  logger.info("Interpolation comlpete.")

  while not os.path.exists(f'{frames_dir}/interpolated.mp4'):
    time.sleep(1)

  if os.path.isfile(f'{frames_dir}/interpolated.mp4'):
      os.rename(f'{frames_dir}/interpolated.mp4', f'{frames_dir}/{folder}({run})_interp.mp4')
  else:
      raise ValueError("%s isn't a file!" % file_path)

  interpolation_end_time = time.time()
  interpolation_duration = interpolation_end_time - interpolation_start_time
  #add interpolation duration to stats dictionary
  stats['interpolation_duration'] = interpolation_duration
  with open(f'{batchFolder}/{batch_name}({batchNum})_stats.txt', 'w') as fp:
    json.dump(stats, fp, ensure_ascii=False, indent=4)


### Update Interpolation Observation

In [ ]:

with open("/content/drive/MyDrive/AI/Pop_Diffusion/interpolation_observations.txt") as f:
    full_interpolation_observation_dict = json.load(f)
full_interpolation_observation_dict[batch_name+"*"+str(interpolation_random_number)]["status"]="success"
with open("/content/drive/MyDrive/AI/Pop_Diffusion/interpolation_observations.txt", "w+") as ant: 
    json.dump(full_interpolation_observation_dict, ant, ensure_ascii=False, indent=4)


## 12. Create gist 

In [ ]:
'''
HTTP Reuests has following parameters: 
1)Request URL 
2)Header Fields
3)Parameter 
4)Request body
'''
#!/usr/bin/env python

if should_create_gist:

    import requests
    import json

    GITHUB_API="https://api.github.com"
    #open "/content/drive/MyDrive/AI/youtube/gist_api.json" and get the string from it
    API_TOKEN = '' #this api is in github_api.txt
    with open("/content/drive/MyDrive/AI/gist/gist_api.txt") as f:
        API_TOKEN = f.read().strip()
    #form a request URL
    url=GITHUB_API+"/gists"
    print ("Request URL: %s"%url)
    #print headers,parameters,payload
    headers={'Authorization':'token %s'%API_TOKEN}
    params={'scope':'gist'}
    #with open(settings) as file:
    #   lines = str(file.readlines())
    def create_gist(gist_type):
        
        if gist_type == "prompt_generator":
          with open(f"{batchFolder}/{batch_name}({batchNum})_{gist_type}.txt") as f:
            payload_content = f.read()
        else:
          file=open(f"{batchFolder}/{batch_name}({batchNum})_{gist_type}.txt")
          lines=json.load(file)
          payload_content=json.dumps(lines,indent=4)
        payload={"description":"GIST created by python code","public":True,"files":{name:{"content":payload_content}}}
        #make a requests
        res=requests.post(url,headers=headers,params=params,data=json.dumps(payload))
        #print response --> JSON
        j=json.loads(res.text)
        print('j',j)
        # Print created GIST's details
        print ("Gist URL : %s"%(j['url']))
        print ("GIST ID: %s"%(j['id']))
        gist_URL=str("https://gist.github.com/"+github_username+"/%s"%(j['id']))
        print(gist_type , gist_URL)
        return gist_URL
    settings_gist_URL=create_gist("settings")
    stats_gist_URL=create_gist("stats")
    prompt_generator_gist_URL=create_gist("prompt_generator")

## 13. Get Music

In [ ]:
#check if there are any mp3 in frames_dir
import os
def check_mp3_file():
    mp3_file_exists=False
    for file in os.listdir(frames_dir):
        if file.endswith(".mp3"):
            mp3_file_exists=True
            break
    return mp3_file_exists
mp3_name=""
music_credits=""

if check_mp3_file():
    for file in os.listdir(frames_dir):
        if file.endswith(".mp3"):
            mp3_name=file
            break
    music_credits_filename=mp3_name.replace('.mp3','')+"_credits.txt"
    with open(frames_dir+'/'+music_credits_filename) as f:
        music_credits=f.read()
else:
    !pip install selenium
    !apt-get update # to update ubuntu to correctly run apt install
    !apt install chromium-chromedriver
    !pip3 install  pyvirtualdisplay selenium webdriver_manager  > /dev/null
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
   
    import sys
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.options import Options
    from webdriver_manager.chrome import ChromeDriverManager
    import time
    import random

    if 'http://dig.ccmixter.org' not in ccmixter_search_url_or_keyword:
        ccmixter_search_url_or_keyword="http://dig.ccmixter.org/search?searchp="+ccmixter_search_url_or_keyword

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("window-size=1400,600")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',chrome_options=options)

    driver.get(ccmixter_search_url_or_keyword)
    original_html = driver.page_source
    time.sleep(2)
    #print("original_html", original_html)
    number_of_songs_at_page=str(original_html).split('1 - ')[1].split(' of ')[0]
    print ("number_of_songs_at_page", number_of_songs_at_page)
    x=str(original_html).split('1 - '+number_of_songs_at_page+' of ')[1].split('</div><label')
    number_of_songs = x[0].replace(',','')
    #print("number_of_songs", number_of_songs)
    url_with_all_songs=ccmixter_search_url_or_keyword.replace('search?',"search?limit="+number_of_songs+"&")
    #url_with_all_songs="http://dig.ccmixter.org/search?limit="+number_of_songs+"&searchp=instrumental"
    #new_url="http://dig.ccmixter.org/search?limit=10&searchp=instrumental"
    #print("url_with_all_songs", url_with_all_songs)

    driver.get(url_with_all_songs)
    all_songs_html = driver.page_source
    time.sleep(10)
    y=str(all_songs_html).split('<a class="upload-link song-title" href="/files/')
    song_list = []
    for i in range(1,len(y)): #use split to get all song names
        song_list.append(y[i].split('</a> <a class="people-link artist-name light-color" ')[0].split('/')[1].split('">')[0])
    random_song=random.sample(song_list,1)
    print("random_song: ", random_song)
    random_song_id = random_song[0].split('"')[0]


    #button_element = driver.find_element_by_xpath("//button[@class='btn btn-warning  btn-lg' and @data-reactid='.0.1.6.0.2.0.0.1.$"+random_song_id+".2.0']")

    button_element = driver.find_element(by=By.XPATH, value="//button[@class='btn btn-warning  btn-lg' and @data-reactid='.0.1.6.0.2.0.0.1.$"+random_song_id+".2.0']")
    WebDriverWait(driver,10).until(EC.element_to_be_clickable(button_element)).click()
    driver.switch_to.active_element
    time.sleep(2)
    html_of_song_popup = driver.page_source
    time.sleep(12)
    #open text file
    text_file = open("data.txt", "w")
    
    #write string to file
    text_file.write(html_of_song_popup)
    


    #close file
    text_file.close()
    music_credits=str(html_of_song_popup).split('textarea readonly')[1].split('">')[1].split('</textarea>')[0]
    print(music_credits)



    mp3_url = 'http://ccmixter.org/' + str(html_of_song_popup).split('.mp3" download=""')[0].split('http://ccmixter.org/')[1] + '.mp3'
    print(mp3_url)
    mp3_name = "None"
    if 'content' in mp3_url:
        mp3_name = mp3_url.split('http://ccmixter.org/content/')[1].replace('/','_')
    elif 'contests' in mp3_url:
        mp3_name = mp3_url.split('http://ccmixter.org/contests/')[1].replace('/','_')
    with open(frames_dir+'/'+mp3_name.split('.mp3')[0]+"_credits.txt", "w") as text_file:
        text_file.write(music_credits)
        
    import os
    import requests
    full_save_path = os.path.join(os.getcwd())
    base_path = os.path.dirname(full_save_path)
    if not os.path.isdir(base_path):
        os.makedirs(base_path)

    r = requests.get(mp3_url, headers={'referer': 'http://ccmixter.org/'})
    if r.ok:
        with open(frames_dir+'/'+mp3_name, 'wb') as f:
            f.write(r.content)
    else:
        r.raise_for_status()



## Manual Music Selection 

In [ ]:
# !pip install jp_proxy_widget
# from jupyter_bbox_widget import BBoxWidget
if not automatically_choose_music:
  !pip install ipython_blocking
  import ipython_blocking

import ipywidgets as widgets
from IPython.display import Audio
from IPython.core.display import display
def audio_player():
  display(Audio(frames_dir+'/'+mp3_name,autoplay=True))
# import os
# import json
# import base64


def continue_button_pressed(args):
  print("continur")
def play_music_button_pressed(args):
  print("music ply")
def delete_song_button_pressed(args):
  print("choose")
  global mp3_name
  os.remove(frames_dir+'/'+mp3_name)
  os.remove(frames_dir+'/'+mp3_name.split('.mp3')[0]+"_credits.txt")
  # dir = os.path.dirname(os.path.realpath("_file_"))
  # print(dir)
  # dir = '/content/drive/MyDrive/Colab Notebooks'
  # filename = get('http://

continue_button = widgets.Button(
    description='Continue',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
continue_button

delete_song_button = widgets.Button(
    description='New Song',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Play Music',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
delete_song_button
continue_button.on_click(continue_button_pressed)
# play_music_button.on_click(play_music_button_pressed) 
delete_song_button.on_click(delete_song_button_pressed)
items = [continue_button,delete_song_button]
box = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 600px)"), disabled =False)


if automatically_choose_music:
  box.layout.visibility="hidden"
else:
  box.layout.visibility=="visible"
  audio_player()
box


In [ ]:
if not automatically_choose_music:
  %block continue_button

## 13. Video functions 

In [ ]:
def get_video_length(video_name):
    duration = subprocess.check_output(['ffprobe', '-i', video_name, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
    duration2=str(duration).replace("b'","")
    duration2=str(duration2).replace("\\r\\n'","")
    duration2=str(duration2).replace("\\n'","")
    intduration=round(float(duration2))+1
    return intduration

## 14. Change Speed of a Video

In [ ]:
import subprocess
#vf = f'select=not(mod(n\,{extract_nth_frame}))'

current_video_length=get_video_length(f'{frames_dir}/{folder}({run})_interp.mp4')
length_multiplier=str(finished_video_length/current_video_length)
vf="setpts="+length_multiplier+"*PTS"
# subprocess.run(['ffmpeg', '-i', f'{video}', '-vf', f'{vf}', f'{speed_changed_video_name}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
#   #!ffmpeg -i {video_init_path} -vf {vf} -vsync vfr -q:v 2 -loglevel error -stats {videoFramesFolder}/%04d.jpg

cmd = ['ffmpeg', '-i',f'{frames_dir}/{folder}({run})_interp.mp4', '-vf', f'{vf}', f'{frames_dir}/{batch_name}({batchNum})_speed_changed_output.mp4']

process = subprocess.Popen(cmd, cwd="./", stdout=subprocess.PIPE, stderr=subprocess.PIPE) 
stdout, stderr = process.communicate()
try:
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)

  else:

      print("The video is ready and saved to the images folder")
      #delete all images from label folder

except:
  pass

## 15. Add audio to video

In [ ]:
if not os.path.isfile(f'{frames_dir}/{batch_name}({batchNum})_finished_video.mp4'):
    !pip install ffmpeg-python
    import ffmpeg
    import sys
    duration = subprocess.check_output(['ffprobe', '-i', f'{frames_dir}/{batch_name}({batchNum})_speed_changed_output.mp4', '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
    duration2=str(duration).replace("b'","")
    duration2=str(duration2).replace("\\r\\n'","")
    duration2=str(duration2).replace("\\n'","")
    intduration=round(float(duration2))+1

    print(intduration)
    input_video = ffmpeg.input(f'{frames_dir}/{batch_name}({batchNum})_speed_changed_output.mp4')
    input_audio = ffmpeg.input(frames_dir+'/'+mp3_name, t=intduration)
    #check to see if finished_video.mp4 exists



    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(f'{frames_dir}/{batch_name}({batchNum})_finished_video.mp4').run()
    #how to delete a video file
    os.remove(f'{frames_dir}/{batch_name}({batchNum})_speed_changed_output.mp4')




#subprocess.run(ffmpeg -i "video.mp4" -i "song.mp3" -shortest outPutFile.mp4)

#ffmpeg -i "video.mp4" -i "song.mp3" -shortest outPutFile.mp4



## Upscale Video

In [ ]:
cmd = ['ffmpeg', '-i',f'{frames_dir}/{folder}({run})_finished_video.mp4', '-vf', 'scale=-1:1080:flags=lanczos', '-c:v','libx264', '-crf','21',f'{frames_dir}/{batch_name}({batchNum})_upscaled.mp4']
#ffmpeg -i colortree.mp4 -vf scale=-1:1080:flags=lanczos -c:v libx264 -crf 21 out.mp4
process = subprocess.Popen(cmd, cwd="./", stdout=subprocess.PIPE, stderr=subprocess.PIPE) 
stdout, stderr = process.communicate()
try:
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)

  else:

      print("The video is ready and saved to the images folder")
      #delete all images from label folder

except:
  pass

## 16. Create Youtube Video 

In [ ]:
#!/usr/bin/python
should_upload_video=True #@param{type,'boolean'}
if should_upload_video:
  import httplib2
  import os
  import random
  import sys
  import time

  from apiclient.discovery import build
  from apiclient.errors import HttpError
  from apiclient.http import MediaFileUpload
  from oauth2client.client import flow_from_clientsecrets
  from oauth2client.file import Storage
  from oauth2client.tools import argparser, run_flow


  # Explicitly tell the underlying HTTP transport library not to retry, since
  # we are handling retry logic ourselves.
  httplib2.RETRIES = 1

  # Maximum number of times to retry before giving up.
  MAX_RETRIES = 10

  # Always retry when these exceptions are raised.
  RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError)

  # Always retry when an apiclient.errors.HttpError with one of these status
  # codes is raised.
  RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

  # The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
  # the OAuth 2.0 information for this application, including its client_id and
  # client_secret. You can acquire an OAuth 2.0 client ID and client secret from
  # the Google API Console at
  # https://console.developers.google.com/.
  # Please ensure that you have enabled the YouTube Data API for your project.
  # For more information about using OAuth2 to access the YouTube Data API, see:
  #   https://developers.google.com/youtube/v3/guides/authentication
  # For more information about the client_secrets.json file format, see:
  #   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
  CLIENT_SECRETS_FILE = "/content/drive/MyDrive/AI/youtube/client_secrets.json"

  # This OAuth 2.0 access scope allows an application to upload files to the
  # authenticated user's YouTube channel, but doesn't allow other types of access.
  YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
  YOUTUBE_API_SERVICE_NAME = "youtube"
  YOUTUBE_API_VERSION = "v3"

  # This variable defines a message to display if the CLIENT_SECRETS_FILE is
  # missing.
  MISSING_CLIENT_SECRETS_MESSAGE = "Error"

  VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")

  class ourargs:
    def __init__(self, **kwargs):
      self.__dict__.update(kwargs)

  def get_authenticated_service(args):
    flow = flow_from_clientsecrets(CLIENT_SECRETS_FILE,
      scope=YOUTUBE_UPLOAD_SCOPE,
      message=MISSING_CLIENT_SECRETS_MESSAGE)

    #storage = Storage("%s-oauth2.json" % sys.argv[0])
    storage = Storage("/content/drive/MyDrive/AI/youtube/upload_video.py-oauth2.json")
    credentials = storage.get()

    # if credentials is None or credentials.invalid:
    #   credentials = run_flow(flow, storage, args)

    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
      http=credentials.authorize(httplib2.Http()))

  def initialize_upload(youtube, options):
    tags = None
    if options.keywords:
      tags = options.keywords.split(",")

    body=dict(
      snippet=dict(
        title=options.title,
        description=options.description,
        tags=tags,
        categoryId=options.category
      ),
      status=dict(
        privacyStatus=options.privacyStatus
      )
    )

    # Call the API's videos.insert method to create and upload the video.
    insert_request = youtube.videos().insert(
      part=",".join(body.keys()),
      body=body,
      # The chunksize parameter specifies the size of each chunk of data, in
      # bytes, that will be uploaded at a time. Set a higher value for
      # reliable connections as fewer chunks lead to faster uploads. Set a lower
      # value for better recovery on less reliable connections.
      #
      # Setting "chunksize" equal to -1 in the code below means that the entire
      # file will be uploaded in a single HTTP request. (If the upload fails,
      # it will still be retried where it left off.) This is usually a best
      # practice, but if you're using Python older than 2.6 or if you're
      # running on App Engine, you should set the chunksize to something like
      # 1024 * 1024 (1 megabyte).
      media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True)
    )

    resumable_upload(insert_request)

  # This method implements an exponential backoff strategy to resume a
  # failed upload.
  def resumable_upload(insert_request):
    response = None
    error = None
    retry = 0
    while response is None:
      try:
        print ("Uploading file...")
        status, response = insert_request.next_chunk()
        if response is not None:
          if 'id' in response:
            print ("Video id '%s' was successfully uploaded." % response['id'])
          else:
            exit("The upload failed with an unexpected response: %s" % response)
      except HttpError as e:
        if e.resp.status in RETRIABLE_STATUS_CODES:
          error = "A retriable HTTP error %d occurred:\n%s" % (e.resp.status,
                                                              e.content)
        else:
          raise
      except RETRIABLE_EXCEPTIONS as e:
        error = "A retriable error occurred: %s" % e

      if error is not None:
        print (error)
        retry += 1
        if retry > MAX_RETRIES:
          exit("No longer attempting to retry.")

        max_sleep = 2 ** retry
        sleep_seconds = random.random() * max_sleep
        print ("Sleeping %f seconds and then retrying..." % sleep_seconds)
        time.sleep(sleep_seconds)

  if __name__ == '__main__':
    # argparser.add_argument("--file", required=True, help="Video file to upload")
    # argparser.add_argument("--title", help="Video title", default="Test Title")
    # argparser.add_argument("--description", help="Video description",
    #   default="Test Description")
    # argparser.add_argument("--category", default="22",
    #   help="Numeric video category. " +
    #     "See https://developers.google.com/youtube/v3/docs/videoCategories/list")
    # argparser.add_argument("--keywords", help="Video keywords, comma separated",
    #   default="")
    # argparser.add_argument("--privacyStatus", choices=VALID_PRIVACY_STATUSES,
    #   default=VALID_PRIVACY_STATUSES[0], help="Video privacy status.")
    # args = argparser.parse_args()

    if show_first_prompt_in_youtube_description:
      youtube_description="Created by Pop Diffusion: (https://github.com/tjthejuggler/Pop-Diffusion)\n\n \
        A fork of Disco Diffusion: (https://github.com/alembics/disco-diffusion)\n\n \
        Film Interpolation: (https://github.com/google-research/frame-interpolation)\n\n \
        First prompt:\n"+str(text_prompts[0])+"\n\n \
        Settings:\n"+settings_gist_URL+"\n\n \
        Stats:\n"+stats_gist_URL+"\n\n \
        Prompt Generator:\n"+prompt_generator_gist_URL+"\n\n \
        Music:\n"+music_credits 
    else:
      youtube_description="Created by Pop Diffusion: (https://github.com/tjthejuggler/Pop-Diffusion)\n\n \
        A fork of Disco Diffusion: (https://github.com/alembics/disco-diffusion)\n\n \
        Film Interpolation: (https://github.com/google-research/frame-interpolation)\n\n \
        Settings:\n"+settings_gist_URL+"\n\n \
        Stats:\n"+stats_gist_URL+"\n\n \
        Prompt Generator:\n"+prompt_generator_gist_URL+"\n\n \
        Music:\n"+music_credits
                  
    #put a variable into a text file
    with open(f'{frames_dir}/{batch_name}({batchNum})_youtube_description.txt', "w") as text_file:
      text_file.write(youtube_description)

    args=ourargs(file=f'{frames_dir}/{batch_name}({batchNum})_upscaled.mp4',title=batch_name,description=youtube_description,category="22",keywords="Disco Diffusion, AI Art",privacyStatus="private")
    # args.file=video
    # args.title=name
    # args.description=URL
    


    if not os.path.exists(args.file):
      exit("Please specify a valid file using the --file= parameter.")

    youtube = get_authenticated_service(args)
    try:
      initialize_upload(youtube, args)
    except HttpError as e:
      print ("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))

### Save settings.txt to the settings_files folder

In [ ]:
from IPython.display import display
import ipywidgets as widgets


def clicked(arg):
    save_settings(False)
    print("button has been clicked!")

button_download = widgets.Button(description = 'Save')   
button_download.on_click(clicked)
display(button_download)

#play an mp3 file
def play_mp3(mp3_file):
    import pyglet
    music = pyglet.media.load(mp3_file)
    music.play()
    pyglet.app.run()
    music.stop()
    pyglet.app.exit()
